In [1]:
si_id = [1,6,4,2,5,3,7,8,31,32,33,34,35,36,37,38,39]

In [2]:
#xPath로 게시글 수 가져온 것,, 먹힐까?

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt
from selenium.common.exceptions import NoSuchElementException
import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '관광지'
# 파일 저장
f_dir = 'c:\\캡디_공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EA%B4%80%EA%B4%91%EC%A7%80&choiceTagId=3f36ca4b-6f45-45cb-9042-265c96a4868c')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
view2 = []      # 게시글 내용 컬럼


no = 1

for j in range(0,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()

    time.sleep(3)

    r_cnt= driver.find_element_by_xpath('''//*[@id="totalCnt"]''').text
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)
    
    cnttt = 150
    page_cnt = math.ceil(cnttt/10)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
#             if no > cnttt :
#                 break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    
            con_1 = content_list.find('p').get_text()
            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD = '0'
            except:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
            
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except IndexError:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except NoSuchElementException:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except:
                addr = '0'
            
            
            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            print(con_1)
            f.write(str(no) + ';'+ str(title) + ';'+str(area_CD) +';'+ str(addr) +';'+ str(con_1) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)
            view2.append(con_1)
            
            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            
            if no > cnttt:
                break
        if i > page_cnt+1:
            break
        i += 1
        
        if no > cnttt:
            break
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
    i=0
    k=0
    no=1
        
a = pd.DataFrame()
a['no']=no2
a['title']=contents
a['area_cd']=area_CD2
a['addr']=addr2
a['content']=view2

# csv 형태로 저장하기
a.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
a.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 562 건
실제 최종 출력 건수 150
no 1 :  G밸리 제품전시관
서울 구로구
서울특별시 구로구 디지털로26길 111
4차산업애 맞춰 4가지 체험 (드론, 코딩, VR, 3D 프린팅)과 산업관광 운영 제4차산업체험센터에서는 한 공간에 4개의 제4차산업혁명을 대표하는 기술과 체험을 준비하여 드론, 코딩, VR 3D프린팅의 원리를 이해하고, 기초 실습과 관련 시뮬레이션을 진행하고있다.이를 바탕으로 실제 응용해 볼 수 있는 커리큘럼들이 준비되어 참여자들은 개별체험을 통하여 스스로 심화하고, 이를 다양한 분야에 적용할 수 있는 통찰을 경험할 수 있도록한다.산업관광이란, 기업체의 생산현장이나 홍보시설 등을 활용한 관광 콘텐츠를 말하며, 이를 통해 직간접적인 비즈니스 를 연계하고, 관련 산업의 활성화를 지원하는 새로운 형식의 산업과 관광을 합친 컨텐츠이다.이러한 산업관광을 통해 참가자들 (관광객, 지역주민, 학생 등등)은 산업적 배움과 더불어 재미있는 새로운 볼거리와 체험거리를 경험할 수 있는 기회를, 기업과 국내 및 해외 바이어들은 기업의 우수성을 홍보할 수 있는 기회를 제공한다. 


no 2 :  생명보험교육문화센터
서울 종로구
서울특별시 종로구 새문안로5길 31 센터포인트 B/D 3층
금융보험에 대해 재미있게 체험하고 학습할 수 있는 공간, 생명보험교육문화센터입니다.생명보험교육문화센터는 생명보험의 기본정신인 생명존중, 상부상조를 실천하고자 생명보험사 공동 사회공헌 활동의 일환으로 2014년 6월 개관하였습니다. 약 300평 규모의 생명보험교육문화센터에서는 태블릿PC 등을 활용해 어렵게 느낄 수 있는 금융보험 내용을 쉽게 학습할 수 있고, 미션과 문제를 풀어가는 과정을 통해 자기주도학습(self-directed learning)이 가능합니다. 프로그램은 약 2시간동안 진행되는 체험학습과 4시간동안 진행되는 진로학습이 있으며, 매년 약 4,000명의 학생들이 방문하고 있습니다. 생명보험교육문화센터는 교육부 교육기부대상 및 금감원 금융공모

no 11 :  관악산
서울 관악구
서울특별시 관악구 관악로
관악산(높이 632.2 m)은 1973년 관악구가 영등포구에서 분구되면서, 명산으로서 산 이름이 구의 명칭이 되어 관악구의 상징이자 자랑이 되고 있으며, 관악구 문화유산의 대부분이 관악산에서 비롯되었다. 1968년에 건설부 고시 제34호로 도시자연공원으로 지정되었으며 오늘날에는 수많은 서울 시민들이 즐겨 찾는 휴식처로 서울의 명소가 되었다. 곳곳에 드러난 암봉들이 깊은 골짜기와 어울려 험준한 산세를 이루고 있는 관악산은 산의 규모가 그다지 크지 않고 도심에서 가까워 누구나 하루 일정으로 산에 오를 수 있는데 봄에는 관악산 입구 쪽으로 벚꽃이 만발하고, 철쭉이 필 때는 철쭉제가 열리기도 한다. 봄철에 무리지어 피는 철쭉꽃과 여름의 짙은 녹음과, 계곡 깊은 곳에 동폭포, 서폭포의 물소리가 장엄하고 가을에는 단풍, 겨울에는 설경이 명산 관악의 면모를 자랑하고 있다. 관악산 정상에는 지상 레이다 관측소와 조선 태조 이성계가 서울을 도읍지로 정할 때 연주사와 원각사 두 절을 지어 화환에 대처했다고 하는 정상의 원각사와 연주암을 비롯하여 크고 작은 사찰과 암자가 있는데, 아슬아슬한 벼랑 위에 자리잡고 있는 연주대는 관악산의 모든 등산로가 집결하는 곳이다.


no 12 :  이음피움봉제역사관
서울 종로구
서울특별시 종로구 창신4가길 26
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.28 ~ 별도 안내시까지)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기''이음피움’은 실과 바늘로 천을 이어서 옷을 탄생시키 듯 서로를 잇는다는 의미의 ‘이음’과 꽃이 피어나듯 소통과 공감이 피어난다는 뜻의 ‘피움’을 합해 만들어진 이름이다. 원단을 가득 실은 오토바이, 드르륵 드르륵 빠르게 돌아가는 재봉틀 소리, 하얀 김을 뿜으며 나가는 스팀다리미, 옷이 만들어지는 이 곳에서 봉제 산업의 과거, 현재, 그리고 미래를 이야기한다.창신동은 동대문패션타운의 든든한 배후 생산기지이자 전 세계 몇 남지 

no 27 :  키즈오토파크
서울 광진구
서울특별시 광진구 능동로 216
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관(2020.08.16.~)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기어린이교통안전체험관은 3,000㎡ 부지에 운전 코스와 신호등 등을 배치해 어린이들이 자동차를 운전하며 교통문화를 체험할 수 있는 시설이다. 오토 가상체험시설, 보행교육장, 주행교육장, 교통안전 면허시험장, 오토부스 등 다양한 시설을 갖추고 있다. 입체 동영상을 통해서 가상체험, 안전띠체험, 보행실습 등 다양한 교육프로그램을 체험할 수 있다. 교통안전교육을 수료한 어린이들에게는 시험을 치른 후 어린이 교통안전 면허증를 발급해준다.(3학년 어린이는 4학년이 끝나는 2월까지 면허시험 신청 가능)


no 28 :  수도박물관
서울 성동구
서울특별시 성동구 왕십리로 27
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관(2020.08.19.~)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기뚝섬수원지 제1정수장은 1908년 9월부터 한국 최초로 수돗물을 생산, 공급한 곳이다. 이를 2008년 복원, 정비하여 개관하였으니 100년만이다. 이곳에서 상수도의 역사와 가치를 이해하고 서울시 수돗물‘아리수’생산과정도 살펴본다. 박물관 건물은 근대식 건축물의 특징이 잘 나타나 서울시 유형문화재 제 72호로 지정되었다.


no 29 :  국립민속박물관&국립민속박물관 어린이박물관
서울 종로구
서울특별시 종로구 삼청로 37
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.19 ~ 상황 종료시)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기한국인의 생활문화를 직접 보고 체험하는 살아 있는 교육의 터전국립민속박물관은 우리나라 전통생활양식과 관련된 4,000여 점의 민속자료가 전시된 있는 국내 유일의 민속생활사 관련 국립·종합박물관으로서 경복궁 안에 있다. 민속문화의 연구·수집·보존과 문화교육 및 생활문화 전시로 전통문화에 대한 올바른 인

no 37 :  씨랄라 워터파크
서울 영등포구
서울특별시 영등포구 문래로 164
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관(2020.08.22.~)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기''씨랄라'' 는 영문 ''SEA[:바다]와 의성어 LALA[:신나고 즐거운]''의 합성어로일상에 지친 도시인들에게 지중해 바닷가에 온 듯한 휴양감과 계절, 거리의 제약을 뛰어 넘어 사계절 언제나 가장 가깝게 찾으실 수 있는 도심 속 신개념 Waterpark & Spa 입니다.[출처 : 씨랄라워터파크]


no 38 :  서울 구 벨기에영사관(현, 서울시립미술관 남서울생활미술관)
서울 관악구
서울특별시 관악구 남부순환로 2076
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관(2020.08.19.~)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기1903년 설립하여 1905년에 준공된 이 건물은 일본 북육토목회사에서 시공하고 기사 小玉이 설계했으며 西島가 감독하였다고 한다. 이중화의 경성기략에는 1900년 입경한 벨기에 전권위원 레온 방카르가 1902년 10월 이곳에 자리를 잡아 영사관 건축에 착수하였다고 기록되어 있다. 그 후 1919년에 영사관이 충무로 1가 18번지로 옮긴 다음 이 건물은 일본 횡빈생명보험회사 사옥으로 쓰이다가, 일본 해군성 무관부 관저로 이용되었으며, 해방 후 해군헌병대로 사용되었다.1970년 상업은행이 불하 받아 사용하고 있다가 1982년 8월 현재의 남현동으로 이전되어 한국상업은행(현 우리은행) 사료관으로 사용되었다. 현재는 서울시립미술관 남서울분관으로 새롭게 꾸며져 구민을 위한 문화공간으로 활용되고 있다. 건물은 지하 1층, 지상 2층으로 벽돌과 석재를 혼용하였고, 고전주의 양식의 현관과 발코니의 이오닉 오더(Ionic order) 석주 등은 외장상 상당히 수려한 양식임을 잘 나타내고 있다.* 규모 - 건축면적 1,569.58m², 2층 벽돌구조* 형성시기 - 1905년


no 39 :  놀이똥산
서울 종

no 52 :  남산골한옥마을
서울 중구
서울특별시 중구 퇴계로34길 28
1998년 조성된 남산골한옥마을은 남산 북측 옛 수도방위사령부 부지 총 79,934m²(24,180평)에 한옥 5동, 전통공예관, 천우각, 전통정원, 서울남산국악당, 새천년타임캡슐 광장으로 구성되어 시민과 관광객을 맞이하고 있다. 번화한 도심 한 가운데에서 한국 전통문화를 소재로 한 다양한 체험거리를 만나볼 수 있으며, 남산 자락을 따라 전통조경 양식으로 조성된 계곡과 정자, 각종 화초가 만들어내는 아름다운 풍경을 만날 수 있다. 전통정원 가장 높은 곳에는 서울정도 600년을 기념하고 400년 후 서울정도 천년을 기대하며 1994년 당대의 기억을 매설한 서울천년타입캡슐 광장이 조성되어 있다. 옛 가옥을 복원해 놓은 남산골한옥마을 남산골한옥마을 안으로 들어서면 연못 청학지와 그 곁으로 천우각이 있다. 천우각 광장을 지나 돌계단을 오르면 서울 곳곳에 있던 한옥이 이전·복원되어 옛 사람들의 생활방식을 볼 수 있게 한 예스러운 공간이 나타난다.모두 다섯 채의 한옥으로서 삼각동 도편수 이승업 가옥, 삼청동 오위장 김춘영 가옥, 관훈동 민씨 가옥, 제기동 해풍부원군 윤택영 재실, 옥인동 윤씨 가옥이 있다. 이 중 건물 이전이 불가능할 정도로 낡은 옥인동 윤씨 가옥만 새 자재를 사용해 복원했고 나머지 건물은 종래의 집 부재를 그대로 이전했다. 남산골한옥마을에서 즐기는 다양한 체험거리 남산골한옥마을에는 다양한 체험거리가 마련되어 있다. 한옥 다섯 채를 활용하여 한옥실내 공간을 체험하면서 동시에 한국 전통문화프로그램을 접할 수 있다. 한복 입기, 한지 접기, 한글 쓰기, 전통차 마시기를 체험할 수 있고, 전통예절학교와 한방체험도 운영하고 있다. 토요일과 일요일 양일에는 전통혼례가 실제로 치러지고 전통혼례체험도 운영하고 있다. 그 밖에도 천우각 맞은편 원두막에서는 짚공예 시연이 이루어지고 있으며, 천우각 무대에서는 태권도 시범공연과 체험 행사에 참여할 수 있다. 피금정 마당에서는 제기차기, 윷놀이, 비석

IndexError: list index out of range

In [3]:
#xPath로 게시글 수 가져온 것,, 먹힐까?

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt
from selenium.common.exceptions import NoSuchElementException
import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '관광지'
# 파일 저장
f_dir = 'c:\\캡디_공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EA%B4%80%EA%B4%91%EC%A7%80&choiceTagId=3f36ca4b-6f45-45cb-9042-265c96a4868c')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
view2 = []      # 게시글 내용 컬럼


no = 1

for j in range(4,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()

    time.sleep(3)

    r_cnt= driver.find_element_by_xpath('''//*[@id="totalCnt"]''').text
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)
    
    cnttt = 150
    page_cnt = math.ceil(cnttt/10)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
#             if no > cnttt :
#                 break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    
            con_1 = content_list.find('p').get_text()
            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD = '0'
            except:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
            
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except IndexError:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
                
            except NoSuchElementException:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            
            except:
                addr = '0'
            
            
            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            print(con_1)
            f.write(str(no) + ';'+ str(title) + ';'+str(area_CD) +';'+ str(addr) +';'+ str(con_1) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)
            view2.append(con_1)
            
            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            
            if no > cnttt:
                break
        if i > page_cnt+1:
            break
        i += 1
        
        if no > cnttt:
            break
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
    i=0
    k=0
    no=1
        
a = pd.DataFrame()
a['no']=no2
a['title']=contents
a['area_cd']=area_CD2
a['addr']=addr2
a['content']=view2

# csv 형태로 저장하기
a.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
a.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 122 건
실제 최종 출력 건수 122
no 1 :  사직단
광주 남구
광주광역시 남구 사직길 49
* 신에게 제를 올리던 곳, 사직단(社稷壇) *갑오년(1984) 이후 광주의 여제와 함께 폐지되었으며, 한말에는 사직단을 군사 훈련장으로 사용하기도 했었다. 동물원이 들어서면서 그 자취가 없어졌는데, 1993년 광주시가 사직단을 복원, 1994년 광주 향우회 회장(최기영)에 의해 1백년 만에 사직제가 부활되었다. 사직단은 국토의 신과 오곡의 신, 두 신에게 제사를 올리는 제단이다. 국토와 오곡은 민생의 근본이므로 삼국 시대부터 국태민안을 기원하는 사직단을 설치하여 친히 왕이 나가서 제사를 올렸다. 지방에서는 성주가 제주가 되었으므로 광주 목사가 매년 제사를 주관하였다. 사직단은 종묘와 함께 신성시되어 종사(宗社)라고도 했다. 사직제는 매년 맹춘(정월) 상신일에 기고제(그해 풍년을 비는 제), 중춘(음력6월), 중추(음력 8월) 상무일, 납일(음력 12월 8일)걸쳐, 네 차례 거행되었으며, 가뭄에는 기우제, 비가 많이 오면 ''기정제'', 눈이 오지 에 않으면 ''기설제''를 지냈다. 또한 사직제는 혈제(血祭)라 하여 제수에 돼지머리나 양머리를 날것으로 봉헌했다.* 사직단이 있는 사직공원에 대하여 *사직공원의 터는 원래 사직단이 있던 곳이다. 1894년 제사가 폐지되고 1960년대 말에 사직동물원이 들어서면서 사직단은 헐리고 말았다. 그 뒤 사직단의 본래의 모습을 되찾아야 한다는 여론이 조성되자 1991년 동물원을 우치공원으로 옮기고 사직단을 복원하여 현재에 이르고 있다. 사직공원은 각종 수목 및 화초들의 조경이 잘 되어 있으며 특히 매년 4월 15일을 전후해서 벚꽃이 만발, 광주시는 이 무렵이면 밤에 전등을 가설하여 시민들이 밤에도 벚꽃을 즐길 수 있도록 했다. 지금은 시민들이 가장 즐겨찾는 휴식처로 자리매김하게 되었다.


no 2 :  양림동 펭귄마을
광주 남구
광주광역시 남구 천변좌로446번길 7
양림동 주민 센터 뒤에 펭귄모양의 

no 16 :  신창동 선사유적지
광주 광산구
광주광역시 광산구 신창동
* 철기 시대의 생활상을 보여주는 주요 유물 *신창동 선사유적지는 영산강 유역의 충적대지와 낮은 구릉지대에 위치하고 있다. 유적의 북쪽 1.5km에 월계동 장고분(기념물 제20호)이 있다. 이곳에는 초기 철기시대(기원전 2~1세기)의 늪과 못 터, 토기 가마터, 배수 시설, 집자리, 독무덤 등 고대 농경 생활과 관련된 유적이 집중된 곳이다. 독무덤은 토착 농경민의 어린이 무덤으로 유적의 서쪽 구릉 경사 면에 53기가 분포하고 있다. 독의 기본 형태는 평저난형(平底卵形)으로, 축약된 구연 등은 무문토기의 가장 보편적 형식이다. 출토 유물은 무문토기와 평저장경호(平底長經壺) 등의 토기류, 청동제 칼자루장식, 돌도끼, 돌화살촉, 땅파는 도구로 추정되는 철제편 등이 있다. 늪과 못터는 영산강의 범람에 의하여 형성된 것으로, 모두 10개 층으로 나누어지거나 크게 3개 단위로 구성된다. 출토 유물은 빗 괭이, 나무뚜껑, 굽다리 접시, 검은 간토기 등 목재류(木材類), 토기류(土器類), 칠기류(漆器類), 석기류(石器類)와 탄화미 (炭化米), 탄화맥 (炭化麥), 볍씨, 살구씨, 호도씨, 오이씨 등의 씨앗류, 민물 조개류, 물고기뼈, 짐승뼈 등이 있다.* 철기시대의 토기 연구에 의의가 있는 토기 가마 *토기 가마는 지반(地盤)인 황갈색 석비례층을 파고 만들었으며 가마바닥은 U자형으로 5∼8도의 경사를 이루고 있다. 출토 유물은점토대(粘土帶)토기, 검은간 토기, 대접, 두형(豆形)토기, 굽다리 접시, 시루, 옹형(擁形)토기 등의 토기류와 철도자(鐵刀子) 1점 등이 있다. 구상유구(溝狀遺構)는 토기 가마 북쪽에 있는 두줄이 평행한 도랑과 같은 유구이다. 석비례 암반층을 파고 상.하 2단층으로 이루어져 있으며 유구 안쪽은 V 또는 U자형이다. 출토 유물은 점토대토기, 옹형 토기, 컵형 토기, 토기 뚜껑, 가락 바퀴 등이 있다. 조상유구(條狀遺構)는 모두 5조로 너비 10∼25cm의 길다란 모래띠가 40∼70cm

no 28 :  광주 3·1 만세운동 기념비
광주 남구
광주광역시 남구 백서로 13
* 수피아 여고생의 넋을 기리는 비, 광주 3·1 만세운동 기념비 *광주는 일제강점기 시절 광주학생운동을 시작으로 다양한 독립운동이 펼쳐진 고장이다. 이러한 고장에서 3·1 만세운동이 벌어진 것은 당연한 결과이다. 이러한 광주지역의 3·1 만세운동을 기념하고 당시의 넋을 기리기 위해 만들어진 것이 “광주 3.1 만세운동 기념비”이다.1995년 5월 10일에 건립한 이 기념비는 양림동 256번지 수피아여고 후문 안에 자리 잡고 있다.앞면에는 "광주 3·1 만세운동 기념"이라고 새겨져 있고, 뒷면은 1919년 3월 10일 광주 3·1 만세운동에 앞장섰기 때문에 옥고를 치른 수피아 학생들의 이름을 새겨 길이 기념하고자 했다.* 당시의 넋을 기리는 기념비의 추모시 *기념비에는 박순애, 이태옥, 김양순, 윤혈녀, 김덕순, 조옥희, 이금봉, 하영자, 강화선, 이라혈, 최수향, 김만순, 홍순해 등 23명의 명단이 새겨져 있으며, 옆면은 "역사의 별이 되어" 라는 추모시가 다음과 같이 새겨져 있다.천추에 이름 석자 누구나 남기는가, 기미년 3·1 운동 선두에 서서 구국의 일념으로 충성을 맹세하고 태극기 앞에 두고서 독립만세를 외쳤네. 일제의 총칼 앞에 나라가 침탈 당하자 자유 없는 속박에서 사느니보다 사슬을 끊기 위해 목숨 걸고 싸웠네. 모진 고문을 당하여도 이겨낸 고결한 애국 얼이 역사의 별이 되어 무궁토록 빛나네.* 규 모: 넓이 4㎡ 길이 7m* 형 상: 3·1 운동 때 여학생들이 태극기를 들고 만세를 부르고 있는 형상


no 29 :  충효동 정려비각
광주 북구
광주광역시 북구 충효샘길 7
충장공 김덕령(1567~1596) 장군과 그의 부인 흥양 이씨, 그 형 덕홍과 동생 덕진 일가족의 충, 효, 열을 기리기 위해 충효리 마을 앞에 세운 비각이다. 이것이 여느 문과 다른 것은 이 비각 앞의 현판에 새겨 있는 내용에서 "...1788년 정조가 윤음으로서, 표리비를 세우라" 이르고, 충

no 39 :  쌍암공원
광주 광산구
광주광역시 광산구 첨단중앙로182번길 39
* 호수와 여유, 그리고 쉼이 있는 공간, 쌍암공원 *쌍암공원은 광산구 첨단지구 내 광주과학기술원 앞에 위치해 있다. 한가운데 위치하고 있는 호수 때문에 운치를 더하고 있어, 일명 호수공원으로도 불리기도 한다. 호수주변을 따라 잘 조성된 철쭉꽃 숲들과 소나무숲이 있고 하늘을 향해 내뿜는 부유폭기는 상쾌함과 시원함을 더해준다. 한 달에 한두 번은 도심 속의 작은 예술축제 등 이벤트행사가 열려 분위기를 한껏 돋군다. 호수 속에서는 시커먼 물고기 떼가 오는 이를 반기고 듬성듬성 조화를 이룬 물풀들은 자연미를 한껏 더해준다. 유럽의 여느 공원의 가로등 같이 우아하고 고전미 넘치는 96개의 가로등은 밤의 광원을 환상으로 물들인다.* 가족공원으로 자리 잡은 쌍암공원 *146개의 벤치, 깨끗하고 정갈한 화장실3개소 등 편의시설을 갖추고 있는 쌍암공원에는 어린이들을 위한 조합놀이대 3개소 등이 갖추어져 있어 가족 단위의 공원으로 좋다. 또한 축구장과 농구장, 족구나 피구를 즐길 수 있는 드넓은 잔디광장, 체력단련시설 등이 갖추어져 청소년들에게도 인기가 많다. 혹시나 모를 안전사고도 4명의 관리요원이 상주하여 시민불편을 없애주고 있다. 광산구에서는 주민들의 편의와 쾌적한 공간조성을 위해 왕벗나무 240본을 식재하였으며, 팔각정, 화장실 각1개소와 파고다지붕 18개소를 설치했다. 또한 편리한 음용수 공급을 위해 음수전 3개소를 설치 할 예정이어서 앞으로는 더욱 시민들의 쉼터로 자리 잡을 것으로 보인다.


no 40 :  운천저수지
광주 서구
광주광역시 서구 운천로 165
1951년 마륵동 농경지에 농업용수를 공급하고 재해방지를 위해 축조 주변의 대규모 택지개발(금호,상무지구)로 도시중심에 위치하게 되고, 상류수원 고갈과 오폐수가 유입되어 악취와 해충서식지로 전락하게되어 각종 민원이 유발되던 곳을 ''95년 민선 1기 이정일 청장 취임후 년차별 사업추진으로 하수와 오폐수 유입을 차단하고 맑은물을 공

no 56 :  광주무역회관
광주 광산구
광주광역시 광산구 무진대로 282
1995년 10월에 개관된 것으로 1백 50석 규모로 4개국의 동시통역실 5개와 영상 음향시설을 갖춘 회관이다. 국제회의, 각종 학술제, 세미나 등 주변에 은행들이 들어와 있고 바로 앞 도보로 10여분 거리에 대형마트인 이마트가 있어 쇼핑하기에 좋다. 자동차로 15분 거리에 광주공항과 송정리역이 있다. 또한, 20여분 거리에 광주버스터미널이 있다.


no 57 :  상무시민공원
광주 서구
광주광역시 서구 상무공원로 101
서구 치평동 상무 신도심 아파트 단지내에 자리한 아름다운 공원이다. ’94년 11월 12일 지정되어 각종 운동시설과 아름다운 수경관 연출이 가능한 호수 및 열린광장, 조각공원 등 각종 조경, 편의시설 등을 갖춘 광주에서 최대 규모의 공원이다. 상무시민공원은 인공호수 2개, 광장(39,360㎡), 체육시설(종합운동장, 육상트랙, 테니스장 3, 농구장 1, 배구장 1), 여성발전센터, 환경조형작품(공룡나라외 15종) 등으로 이루어져 있다. 특히 운동시설로 3천470평의 국제규격 잔디축구장 및 6,000여 명을 수용할 수 있는 잔디스탠드, 맨발 조깅이 가능한 4백m 우레탄 트랙 등이 설치돼 있는 종합운동장과 소운동장이 있다.조각공원은 상무시민공원내 호숫가를 중심으로 펼쳐져 있다. 각 작품마다 경관 조명시설을 설치해 야간에도 환상적이고 아름다운 작품을 감상할 수 있으며 누구나 만지고 체험하면서 감상할 수 있도록 함으로써 ‘보는 즐거움’과 ‘체험의 즐거움’을 동시에 안겨준다. 휴먼파크를 주제로 조각가 18명의 작품 22점이 설치되어 있다. ‘공룡나라’는 익룡21, 아기공룡, 티라노, 잃어버린 세계 등이 테마로 꾸며진 곳으로 티라노 미끄럼틀을 타며 뛰노는 어린이들의 모습이 눈부시다.


no 58 :  광주 서구 8경
광주 서구
광주광역시 서구 동하길 10
광주광역시 서구에는 자연풍광과 문화유산이 어우러져 다양한 볼거리를 가지고 있다. 그 중에서도 멋있는 풍경이 있으니 그것을 

no 68 :  무등산 주상절리대
광주 동구
광주광역시 동구 용연동 산354-1
주상절리는 용암이 식을 때 수축되어 생기는 절리 중에 단면의 형태가 오각형이나 육각형의 기둥모양인 것을 말한다. 무등산 주상절리는 약 7천 만 년 전에 형성된 것으로 서석대, 입석대, 규봉이 대표적이다. 입석대, 규봉은 풍화가 많이 진행되어 기둥모양이지만 서석대는 풍화가 덜 진행되어 병풍모양을 하고 있다. 정상을 중심으로 산비탈에 있는 너덜겅은 이러한 돌기둥이 무너져 쌓인 것이다. 서있는 바위나, 너덜겅들은 암석의 생성과 풍화과정을 살펴볼 수 있는 희귀한 자연유적이므로 서석대와 입석대를 천연기념물로 지정하였다.* 시대 - 선사시대* 수량/면적 - 107,800㎡(지정구역)


no 69 :  금호타이어 광주공장
광주 광산구
광주광역시 광산구 어등대로 658
* 승차감과 안전의 비법을 찾아라 *자동차의 신발이라 하는 타이어, 타이어가 편안해야 주행이 안전하다. 금호타이어는 세계 180여 개국에 수출하는 세계 10대 타이어메이커로 대 소비자 커뮤니케이션에 적극적이다. 광주공장에서는 타이어 역사관과 함께 공장시설 견학 프로그램을 운영한다.


no 70 :  광주 송정동 떡갈비 골목
광주 광산구
광주광역시 광산구 광산로29번길
광주 송정동에는 20여 개의 떡갈비 전문점이 모인 골목이 있다. 송정동 떡갈비 골목의 음식점들은 저마다 고소하고 쫄깃한 떡갈비 맛으로 손님들의 발길을 끌고 있다. 평일과 주말을 가리지 않고 관광객이 찾아들고, 호남 지역을 방문한 외국인 관광객이라면 반드시 한 번은 들러 가곤 한다. 송정 떡갈비는 여러 가지 면에서 보통의 떡갈비와 다르다. 보통 떡갈비는 소고기를 이용하지만, 이곳은 소고기와 돼지고기를 섞어 만든다. 원래 송정 떡갈비도 소고기를 이용하다가 1990년대 후반 경제가 어려워지면서 재료비가 인상되자, 음식 값을 올리지 않기 위해 돼지고기를 섞어 메뉴로 내놓았다고 한다. 육질이 서로 다른 돼지고기와 소고기가 만나 환상의 궁합을 만들어내며 송정 떡갈비가 탄생한

no 82 :  (전)광주 성거사지 오층석탑
광주 남구
광주광역시 남구 천변좌로338번길 7
* 안정미와 수려함이 넘치는 고려시대 탑, 광주서오층석탑(光州西五層石塔) *광주공원 안에 거북머리와 같이 돌출된 지형 위에 세워져 있다. 탑의 상륜부는 모두 없어졌으나, 전체적인 상태는 매우 양호한데, 통일신라시대의 이중기단 양식으로 변화된 모습을 보여 준다. 탑의 오층 몸체부 중 초층을 다섯 개의 돌로 짜맞춘 것이 다른 탑에서는 흔히 볼 수 없는 독특한 특징이며, 지붕돌의 추녀와 몸체부의 알맞은 비례, 상층으로 올라가면서 줄어드는 각층의 비율이 크지 않아 전체적으로 높게 보이면서도 튼튼한 안정미와 수려함이 넘치는 고려 시대 탑으로서, 뛰어난 수작이라 할 수 있다.* 고려시대 석탑의 모습을 보여주는 탑의 구조 *광주서오층석탑의 기단부는 통일신라 시대의 탑이 대체로 2층기단으로 되어있는 것과는 달리 고려시대 탑에서 흔히 볼 수 있는 것처럼 단층으로 간략화되고 있다. 탑신부는 1층의 옥신석만 빼고 각층마다 옥신석과 옥신석이 모두 한 개의 돌로 만들어졌는데, 각층의 체감률이 적어 탑의 인상이 날렵하고도 높게 보인다. 1층 옥신석은 2단의 별석으로 짰는데 하단은 4매의 돌을 깎아 맞추었고 상단은 한 개의 돌을 다듬어 만든 이형탑이다. 이런 형식은 고려시대의 탑에서만 볼 수 있다.옥개석은 추녀가 넓은 편이며 추녀 네 귀의 위아래가 모두 뒤쪽으로 되바라지도록 전각을 주고 있고, 층받침은 고려탑의 정형대로 모두 4매이다. 옥개석은 받침마다 모서리에서 10㎝씩의 턱을 주어 체감에 균형미를 주고 있다. 2층 옥신석도 옥개석과 마찬가지로 1층 옥신보다 높이가 현저하게 줄고 있다. 탑신 위에 있었던 상륜부는 모두 상실되었다. 이 탑은 1961년에 해체, 보수했는데, 이때 2층 옥신석의 위쪽에 사리공이 발견되었고, 그 속에서 원형이 잘보존된 금동전각형 사리 장엄구를 찾아냈다. 이 사리구는 현재 국립광주박물관에 보관되어 있다.* 탑과 관련되 전설 *탑이 있는 곳은 원래 성거산 또는 성구강이라

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]"}
  (Session info: chrome=85.0.4183.102)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.18363 x86_64)


In [4]:
#xPath로 게시글 수 가져온 것,, 먹힐까?

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt
from selenium.common.exceptions import NoSuchElementException
import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '관광지 광주 다음부터'
# 파일 저장
f_dir = 'c:\\캡디_공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EA%B4%80%EA%B4%91%EC%A7%80&choiceTagId=3f36ca4b-6f45-45cb-9042-265c96a4868c')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
view2 = []      # 게시글 내용 컬럼


no = 1

for j in range(5,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()

    time.sleep(3)

    r_cnt= driver.find_element_by_xpath('''//*[@id="totalCnt"]''').text
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)
    
    cnttt = 150
    page_cnt = math.ceil(cnttt/10)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
#             if no > cnttt :
#                 break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    
            con_1 = content_list.find('p').get_text()
            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD = '0'
            except:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
            
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except IndexError:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
                
            except NoSuchElementException:
                try:
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                except NoSuchElementException:
                    addr = '0'
            except:
                addr = '0'
            
            
            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            print(con_1)
            f.write(str(no) + ';'+ str(title) + ';'+str(area_CD) +';'+ str(addr) +';'+ str(con_1) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)
            view2.append(con_1)
            
            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            
            if no > cnttt:
                break
        if i > page_cnt+1:
            break
        i += 1
        
        if no > cnttt:
            break
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
    i=0
    k=0
    no=1
        
a = pd.DataFrame()
a['no']=no2
a['title']=contents
a['area_cd']=area_CD2
a['addr']=addr2
a['content']=view2

# csv 형태로 저장하기
a.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
a.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 88 건
실제 최종 출력 건수 88
no 1 :  왕버드나무군락지
대전 대덕구
대전광역시 대덕구 미호동
문화체육관광부, 한국관광공사 지정 사진찍기 좋은 녹색명소인 왕버드나무 군락지는 일출과 일몰시 햇살과 물안개가 어우러져 환상적인 풍경을 자아내는 곳이다. 특히, 이곳의 왕버드나무와 어우러진 새벽 물안개와 반영을 담은 풍경은 무릉도원을 연상 시킬 정도로 아름답다. 영화‘ 역린’의 촬영장소로도 유명하다. 썩은 버드나무의 원줄기는 캄캄할 때 빛이 난다하여, 시골사람들은 이것을 도깨비불이라고 하며 무서워 하기도 하였다. 흔히, 산골에서 도깨비가 나온다고 알려진 곳은 습지에서 버드나무가 무성한 숲인 경우가 많다. 수피는 내버들과 더불어 수렴제, 해열제, 이뇨제로도 사용된다.<출처 : 충청북도청 관광항공과>


no 2 :  추동인공생태습지
대전 동구
대전광역시 동구 천개동로 41
추동인공생태습지는 대전시가 2008년 12월26일 습지보호구역으로 지정한 곳으로 멸종 위기종인 천연기념물 제330호 수달, 천연기념물 제327호 원앙 그리고 말똥가리, 흰목물떼새, 맹꽁이 등이 서식하고 있다.슬로문화 속 떠오르는 여행지 ''추동생태습지공원'' 습지에 서식하는 동식물, 미생물과 습지를 구성하는 토양 등은 주변으로부터 흘러나오는 각종 오염된 물을 흡수하여 오염물질을 정화시키고 깨끗한 물을 흘려보내는 자정역할을 하는 것으로 알려져 있으며, 여러 종류의 생물들이 모여살기 때문에 생물의 다양성이 높아 생태적으로 중요한 기능을 담당하기도 한다. 동구의 대청호 둘레길 중 6코스 길은 습지가 발달한 생태관광의 명소로 이어진다.


no 3 :  티놀자 애니멀 파크
대전 유성구
대전광역시 유성구 대덕대로989번길 9-51
※ 코로나바이러스감염증-19 공지사항※ 내용 : 평일휴장/주말 정상운영 (2020.08.23 ~ 별도 안내시까지)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기티놀자 애니멀파크는 동물교감을 통한 인성, 감성의 향상과 디지털체험을 통한 미래지향형

no 14 :  대전시민천문대
대전 유성구
대전광역시 유성구 과학로 213-48
대전시민천문대는 2001년 5월 3일 지자체 1호 천문과학관으로 개관하여 매년 10만 명 이상 천문대를 방문하고 있다. 3층 관측실에서는 천체망원경으로 주간 태양의 홍염, 흑점 관측체험, 야간 다르 행성, 항성, 성운, 성단까지 관측 프로그램 체험과, 1층 천체투영관에서는 사계절 별자리 설명과 천문 관련 영상물 상영, 그리고 "토요 별음악회", "금요 별음악회", "별빛 속에서 시와 음악회"등 천문우주과학과 공연예술 프로그램을 진행하고 있으며, 1층 복도에서는 "아스트로 갤러리"미술 전시전이 진행된다.2017년에는 천문공원 조성, 주차장 설치, 관람객 안전을 위한 진입로 포장과 인도(데크)설치, 보조관측실에 슬로프를 설치하여 장애우도 천체관측을 할 수 있도록 시설보완하였으며, 포토존 등을 새롭게 단장하였다.


no 15 :  구봉산(대전)
대전 서구
대전광역시 서구 상보안윗길 160
* 도끼가 썩는 줄도 모르는 절경, 구봉산 * 높이 264.1m인 구봉산은 서구 관저동, 가수원동,  괴곡동, 흑석동, 봉곡동에 둘러싸여 길게 서 있으면서 아홉 개의 봉우리가 모두 제각기  멋을 자랑한다. 또한 봉우리가 가지런히 수려한 모습으로 솟아 있어 마치 병풍에  그려진 동양화를 보는 듯하다. 이 산의 남쪽에는 갑천이 휘돌아 흐르고 있으며, 바위  벼랑을 이루어 가파르기는 하나 1993년에 건립한 구각정 전망대에서 보면 서구관내가  한눈에 조망되어 참으로 아름다운 경관을 자랑하고 있다.  서쪽 덕고개 마루에서 비재까지 아홉 개의 암봉이  이어져 있다 하여 붙여진 이름이다. 산의 경관이 아름다워 신선이 내려와 살았다는  신선바위는 전망이 좋아 시내는 물론이고 도덕봉(534m), 금수봉(532m)과 멀리 계룡산(845m)이  조망된다.  이 구봉산(九峯山)은 높은 봉우리가 아홉개가 빼어나게  솟아 있다 하여 구봉산이라 표기하고 있으나, 옛날 여지도서에는 구봉산(九鳳山)으로  기록되어 있다. 즉, 구

no 30 :  한국생명공학연구원
대전 유성구
대전광역시 유성구 과학로 125
생명, 그 소중함에 대하여 이곳에서는 생명 현상에 관한 기초 연구와 바이오신소재 등 첨단생명공학을 연구한다. 홍보관에서 게놈프로젝트, 바이오칩, 진단키트, 실험동물의 계통 보존시설, GMO 등에 대해 둘러보고 이해하며 앞으로의 전망, 우려, 실용 사례 등에 대한 강의도 듣는다. 


no 31 :  내원사(대전)
대전 서구
대전광역시 서구 배재로197번길 200
* 도심 속의 작은 절집, 내원사 *부처님의 법력이 멀리 인간세까지 미쳐 중생을 구제할 수 있도록, 도심 속에서도 세상사의 번뇌와 속세의 때를 맑게 정화해주는 사찰이 있다. 대전에도 시내에서 그다지 멀지 않은 야트막한 도솔산이 있다. 도솔산은 인간의 눈높이로 볼 때 산은 높지 않지만 산의 높이와 부처님의 법력이 일치하는 것은 아닌 것이다. 낮은 산이지만 산으로서의 아름다움을 닮은 도솔산에 정신을 맑게 해주는 부처님의 집이 있다. 내원사라는 이름의 사찰은 여러 곳에 마련되어 있는데, 도솔산의 내원사 또한 산사를 처음으로 찾아가는 우리들의 기대를 저버리지 않고 반갑게 맞아주는 곳이다. 빼어난 높은 산에서 상상할 수 없는 포근하고 안정된 느낌이 대전 도솔산의 정기이고, 그 산세를 거스르지 않고 팍팍한 도시 생활에 지친 도시민의 시름을 가만가만 위로하고 풀어줄 수 있을 것 같다.알맞게 자리잡은 대웅전과 대적광전이 주위 풍광과 조화를 이루고 있으며, 등산길에 오고가는 이들이 쉼터로 들러서 잠시 쉬면서 사찰의 여러 곳과 정겨운 마음의 대화를 나눠도 좋을 곳이다.도심 속의 작은 절집 내원사. 그러나 절집이 작다고 내원사가 작은 곳은 결코 아니다. 법신불인 비로자나불이 계셔서 부처님의 대자대비 광명의 불빛이 사방으로 퍼져나가고 있으며, 서방 극락세계를 주재하는 아미타불의 설법이 목탁소리를 따라 불경으로 절집을 감싸고 있고, 사바 세계의 위대한 영웅인 석가모니불상이 이곳을 찾은 이들의 마음을 위로하고 있기 때문이다. 부처님은 멀리 있는 것이

no 45 :  김정선생묘소일원
대전 동구
대전광역시 동구 회남로 117
조선 중종 때 형조판서(刑曹判書)겸 예문관제학을 지낸 충암 김정(沖菴 金淨:1486∼1521) 선생과 관계된 유적이 자리한 곳이다. 선생은 조광조와 더불어 지치주의를 실현시키기 위해 미신타파와 향론의 상호부조에 힘썼고, 향약(鄕約)을 전국에 걸쳐 시행하는데 큰 업적을 남겼다. 기묘사화(1519) 때 조광조 등과 함께 감옥에 갇혔다가 금산에 유배된 후 제주도에서 사약을 받았다. 1978년 대청댐 수몰로 물에 잠긴 대덕군 동면 내탑리에서 이곳으로 묘를 옮기면서 신도비(神道碑), 충암선생의 위패(位牌)를 봉안한 별묘(別廟), 산해당(山海堂) 그리고 그의 부인의 정려각(旌閭閣) 등도 함께 옮겼다. 대청댐 수몰지구에 들어가게 되어 국비로 선생의 묘소 등 유적이 동구 신하동 묘골로 옮겨지게 되었고, 이 지역은 대덕군 지역이던 곳이 근래 대전 광역시 동구로 편입되었다.


no 46 :  대전 회덕 동춘당
대전 대덕구
대전광역시 대덕구 동춘당로 80
조선 효종 때 대사헌, 이조판서, 병조판서를 지낸 동춘당(同春堂) 송준길(1606∼1672) 선생의 별당으로 보물 제209호에 지정된 문화재이다. 늘 봄과 같다는 뜻의 동춘당은 그의 호를 따서 지은 것으로 이곳에 걸린 현판은 송준길 선생이 돌아가신 지 6년째 되는 해인 숙종 4년(1678)에 우암 송시열이 쓴 것이다. 이 공원에는 연못 3개소와 민속 그네, 팔각정과 화장실, 간이체육시설, 벤치, 음수대 등으로 꾸며져 있다. 그리고 매년 이곳에서는 갖가지 문화행사가 열린다. 4월에 열리던 동춘당 문화제는 2011년부터 10월에 구민 축제를 겸해서 열리고 있다. 동춘당 문화제는 동춘당 유학학술세미나, 숭모 제례, 휘호 대회, 한시백일장, 투호 놀이, 그네뛰기 등 우리 고장 유학자인 동춘당 송준길 선생의 학풍과 인격을 기리고, 주민화합과 일체감을 조성하기 위한 것이다.


no 47 :  보문산공원
대전 중구
대전광역시 중구 보문산공원로 446
* 역사와 자연

no 59 :  찬샘마을
대전 동구
대전광역시 동구 냉천로 703
* 정이 샘솟는, 찬샘마을 *찬샘마을은 대청의 맑은 물, 맑은 공기 속에 노고산의 정기를 받아 동쪽에는 수몰민의 마을 내천과 양구대 마을이 있고 서쪽에서는 찬샘마을의 수려한 경관을 배경으로 자연 훼손이 전혀 없는 청정 자연 그대로 보존되어 있는 마을이다. 찬샘마을은 자연 속에서 오감을 일깨우며 마을공간과 자원을 활용한 주제중심통합접근법으로 농사, 생태, 공예, 식문화 체험과 테마를 구성하여 학년별 교과학습 눈높이에 맞는 프로그램을 진행하여 학교에서 접해보지 못했던 것을 체험을 통해 교육으로 이루어내는 교육 농장이다.학습시설은 나비, 장수풍뎅이, 동물농장, 개구리, 자연공작실, 식문화학습관, 감자밭, 고구마밭, 벼학습장, 포도밭, 매실, 복숭아밭과 숲 놀이터가 조성되어 있다.* 찬샘마을의 또 다른 볼거리 *찬샘마을에는 대전광역시 기념물 제19호 노고산성, 제29호 성치산성이 있고 노고산 정상에서 내려다보면 대청호의 경관이 일품이며 주변에 유명한 계족산성과 직현성 그리고 마산동 산성, 견두산성과 대청댐 건너편에 보이는 백골산성을 한눈에 볼 수 있다.


no 60 :  가양비래근린공원
대전 대덕구
대전광역시 대덕구 비래동
가양비래근린공원은 약 42만평으로 대전광역시의 동부권에 위치한 주로 산림 형태의 공원으로 남간정사가 있는 우암사적공원과 경부고속도로 대전육교(비래동)주변의 중앙광장이 조성되어 있는 곳으로 이용권이 양분되어 있다. 특히, 우암사적공원은 낮은 야산 기슭의 숲이 우거진 골짜기를 배경으로 하여 위치하고 있어 대전시민은 물론, 외지인들의 이용객이 날로 증가하고 있다.


no 61 :  대흥동 문화예술의거리
대전 중구
대전광역시 중구 대흥동
* 문화예술의 거리로 다시 태어나다, 대흥동 원도심 *대전 중구 대흥동 일대는 대전의 원도심이다. 세련된 도시 이미지가 느껴지는 건물과 카페가 있는가 하면, 1970~80년대를 떠올리게 하는 손때 묻은 풍경이 공존한다. 대흥동을 멋스럽게 꾸미는 것은 낡

no 77 :  한밭수목원
대전 서구
0
도심 속의 한밭수목원은 정부대전청사와 과학공원의 녹지축을 연계한 전국 최대의 도심 속 인공수목원으로 각종 식물종의 유전자 보존과 청소년들에게 자연체험학습의 장, 시민들에게는 도심 속에서 푸르름을 만끽하며 휴식할 수 있는 공간 제공을 목적으로 조성했다. 한밭수목원의 총 조성면적 387천㎡은 4단계로 구분 연차별로 조성하였으며, 서원(시립미술관 북측)과 남문광장은 2005년 4월 28일 개원하였고, 목련원, 약용식물원, 암석원, 유실수원 등 19개의 테마별 園으로 구성된 동원(평송수련원 북측)은 2009년 5월 9일 개원하였다.* 문의 : 042-270-8452* 개장일 : 하절기(4월~9월) 동원:06시~21시 / 서원:동일 / 열대식물원:09시~18시     동절기(10월~3월) 동원:08시~19시 / 서원:동일 / 열대식물원:09시~18시        ※ 입장은 폐장시간 1시간전까지 가능* 휴무일 : 동원 - 월요일 / 서원 - 화요일 / 열대식물원 - 월요일* 관련 홈페이지 : http://www.daejeon.go.kr/gar/index.do◎이용안내- 이용요금 : 무료- 화장실 : 있음(남녀구분)- 장애인 편의시설 : 장애인화장실 있음(남녀구분) / 장애인주차장 10대- 주차시설 : 있음


no 78 :  대청호 추동습지
대전 동구
0
추동인공생태습지는 대전시가 2008년 12월26일 습지보호구역으로 지정한 곳으로 멸종 위기종인 천연기념물 제330호 수달, 천연기념물 제327호 원앙 그리고 말똥가리, 흰목물떼새, 맹꽁이 등이 서식하고 있다. 슬로문화 속 떠오르는 여행지 ''추동생태습지공원'' 습지에 서식하는 동식물, 미생물과 습지를 구성하는 토양 등은 주변으로부터 흘러나오는 각종 오염된 물을 흡수하여 오염물질을 정화시키고 깨끗한 물을 흘려보내는 자정역할을 하는 것으로 알려져 있으며, 여러 종류의 생물들이 모여살기 때문에 생물의 다양성이 높아 생태적으로 중요한 기능을 담당하기도 한다. 둥구의 대청호 둘레길 중 6

no 6 :  선바위(입암)
울산 울주군
울산광역시 울주군 범서읍
경상북도 청도(淸道)의 억산(億山)에서 시작한 태화강은 고헌과 신불산에서 흐르는 물을 언양에서 아울러 동으로 흘러돌아 넓내(泗淵)에서 또 치술령에서 시류하는 대곡천 물을 합쳐 북으로 흐르는 듯 동류하여 범서 망성(望星)에 이른다. 옛날 신라때 왕이 친공(親供)하는 재(齋)에서 남루한 천인으로 희롱받은 한 중의 발자취를 따라 오다 이곳에서 영축산으로 나르는 듯 사라지는 것을 보고 그가 문수보살임을 깨달은 경순왕이 멀리 바라 보기만 하였다는 "望聖"이란 그곳이다.강물은 또 여기에서 국수봉에서 흐르는 중리천을 합쳐 동남으로 돌아 층암절벽의 높은 벼랑을 받아 남으로 흐르면 여기가 이름높은 백룡담(白龍潭)이다. 옛날 선인들의 말을 빌면 백룡이 자리잡고 살았다는 곳이다. 그리하여 날이 가물어 천지가 타오를 때 이곳에서 머리숙여 기우제를 지내면 문득 영검이 있었다.검은 듯 푸른 수면에 산인가 바위인가 하늘에 솟은 층암, 이것이 시인묵객들이 즐겨찾던 입암(立巖)이다.  물맑고 산좋은 이곳에 하늘은 멋진 조각의 솜씨를 남겨 향인들이 시읊고 노래부르고 자연을 즐기며 정각을 세우니 이름지어 입암정이라 하였다. 오늘도 입암은 몇만년 흘러온 태화강의 증인처럼 우뚝 서있고, 찾는 사람들의 발길은 멈추지 않는다.


no 7 :  회야댐 생태공원
울산 울주군
울산광역시 울주군 웅촌면 통천리 산109-1번지
울산의 회야댐 생태공원은 ''올해의 숨은 관광지''로 선정된 곳이다. 한정개방관광지로 선정된 회야댐 생태습지는 댐 상류지역의 수질개선을 위해 조성된 자연 친화적인 인공습지로 연꽃이 피는 7월 중순부터 8월 중순까지 한시적으로 개방된다. 탐방로는 통천초소안 만남의 광장에서 생태습지까지 왕복 4km를 오가는 코스로 구성되어 있으며, 생태해설사의 설명을 들으며 어른 걸음으로 2시간가량 소요된다. 상수원 보호구역 내 수질보호 등을 위해 탐방 인원은 오전/오후 각 50명으로 인원을 제한한다.


no 8 :  울기등대
울산 동구

no 24 :  울산 문화의거리
울산 중구
울산광역시 중구 옥교동
갤러리, 아뜰리에, 소극장 등으로 이루어진 거리, 예술가들이 자유롭게 창작활동을 하고, 자신들의 작품을 전시, 판매하며 시민들과 소통하는 오픈 갤러리, 울산 중구의 대표 브랜드인 [울산큰애기]의 플래그십 스토어인 [울산큰애기 하우스]와 예술가들의 창작 활동을 지원하는 [종갓집 예술 창작소]가 소재, 2020년에는 문화의거리 일원에 [울산시립미술관]이 완공될 예정이다.


no 25 :  선암호수공원
울산 남구
울산광역시 남구 선암호수길 104
공업 용수공급을 위해 설치되어 수질보전과 안전을 이유로 1.2㎢의 유역면적 전역에 철조망이 설치되어 타지역에 비해 상대적으로 낙후됨으로써 사람이 찾지 않는 소외지역으로 전락한지 40년. 자연과 인간을 경계 지어온 지역을 철조망을 철거하고 저수지 주변의 수려한 자연경관을 활용하여 산책로 및 다양한 테마를 가진 시설물를 조성하였다.


no 26 :  편백산림욕장
울산 북구
울산광역시 북구 달천동 산 175
달천동 천마산(해발 236m)은 편백나무 5ha, 소나무 33ha, 잣나무 2ha가 조성되어 있어 주민휴식처로 각광받고 있다. 주민들이 쉴 수 있는 원두막과 피크닉테이블, 숲 해설판, 평상 등이 설치되어 있고 주변에는 천마산 등산로를 기준으로 솔 숲길(0.45km)과 성터옛길(at), 0.4km 만석골 저수지 및 생태수변 전망데크, 천마산 정상 전망대 등이 마련되어 있다.


no 27 :  신명·정자해변
울산 북구
울산광역시 북구 동해안로 1801
울산의 바다를 끼고 있는 주전에서 출발하여 잘 포장된 해안도로를 타고 20분 정도 북쪽으로 올라가면 강동 정자포구가 나온다. 마을 어귀에 들어서면 입심좋은 아낙들과 매운탕거리나 횟거리를 흥정하는 사람들의 정겨운 풍경과 살아서 파닥거리는 싱싱한 해산물을 눈동냥하여 다니다보면 사람사는 풍경도 재미있지만 이곳 특유의 멸치회 별미를 맛볼 수 있는 기회를 제공하여 주기도 한다. 저녁이면 만선인 배가 포구에 들어오면 갑자

no 41 :  한방테마파크 초락당
울산 울주군
울산광역시 울주군 두동면 계명로 44-34
※ 힐링캠프와 초락당 하우스 이용 문의는 한의원으로 문의초락당(艸樂堂)은 한방의료를 바탕으로 하여 실시되는 한방헬스투어 전문의료기관으로서 2001년도 보건복지부 산하 한국보건산업진흥원 평가 『Health Tour to Korea』로 지정된 바 있다. 의.식.주, 의료, 휴식을 한자리에서 체험 할 수 있는 곳이며 휴양과 치료를 병행하는 곳이다. 옛날 우리의 시골집과 같은 소박함과 자연친화적이고 전원적인 분위기는 환자의 치료율 향상에 도움을 주며 정서적인 면도 배려한 진정한 휴식공간이다.* 진료과목 *부인과, 신경정신과, 이비인후과, 사상체질과, 내과, 침구과, 소아과


no 42 :  현대자동차 울산공장
울산 북구
울산광역시 북구 염포로 700
세계 최대, 자동차가 쑥쑥 탄생하는 곳현대자동차는 세계 최대 규모의 단일 공장이다. 5개의 독립된 공장설비가 있으며 32,000여명이 하루 평균 5,800여대의 차량을 생산한다. 놀라운 것은 60만 그루의 조경수, 말 그대로 숲 속의 공장이라 할 수 있겠다. 견학 프로그램은 생산 공장과 76,000 톤 급의 선박을 접안 할 수 있는 부두 견학 등으로 이루어진다.


no 43 :  간절곶 등대
울산 울주군
울산광역시 울주군 서생면 간절곶1길 39-2
새로운 천년의 첫해가 한반도에서 가장 먼저 떠오르는 곳, 울주군 서생면 대송리의 간절곶은 동해안에서 맨 먼저 떠오르는 해를 맞이할 수 있다. 영일만의 호미곶보다도 1분 빠르게,  강릉시의 정동진보다도 5분이나 더 빨리 해돋이가 시작되는데 그 광경이 장관이다. 울산지역의 진하해수욕장, 서생포 왜성과 함께 새로운 명소로 주목을 받고 있다. 간절곶등대는 울산남부순환도로 입구에서 20여 분 거리에 있다. 아카시아가 필 때 울산에서 부산해운대 쪽으로 해안선을 따라 달릴 때 바다내음보다 아카시아 향기가 더 그윽하다. 하얀 꽃을 주렁주렁 매단 아카시아 줄기가 축축 늘어져 있는 산길을 끼고 달리다가

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="contents"]/div[2]/div[1]/ul/li[4]/div[2]/div/a"}
  (Session info: chrome=85.0.4183.102)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.18363 x86_64)


In [2]:
#xPath로 게시글 수 가져온 것,, 먹힐까?

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt
from selenium.common.exceptions import NoSuchElementException
import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '관광지 울산부터'
# 파일 저장
f_dir = 'c:\\캡디_공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EA%B4%80%EA%B4%91%EC%A7%80&choiceTagId=3f36ca4b-6f45-45cb-9042-265c96a4868c')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
view2 = []      # 게시글 내용 컬럼


no = 1

for j in range(6,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()

    time.sleep(3)

    r_cnt= driver.find_element_by_xpath('''//*[@id="totalCnt"]''').text
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)
    
    cnttt = 150
    page_cnt = math.ceil(cnttt/10)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
#             if no > cnttt :
#                 break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    
            con_1 = content_list.find('p').get_text()
            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD = '0'
            except:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
            
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except IndexError:
                try:
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                except IndexError:
                    addr = '0'
                except NoSuchElementException:
                    addr ='0'
            except NoSuchElementException:
                try:
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                except NoSuchElementException:
                    addr = '0'
            except:
                addr = '0'
            
            
            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            print(con_1)
            f.write(str(no) + ';'+ str(title) + ';'+str(area_CD) +';'+ str(addr) +';'+ str(con_1) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)
            view2.append(con_1)
            
            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            
            if no > cnttt:
                break
        if i > page_cnt+1:
            break
        i += 1
        
        if no > cnttt:
            break
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
    i=0
    k=0
    no=1
        
a = pd.DataFrame()
a['no']=no2
a['title']=contents
a['area_cd']=area_CD2
a['addr']=addr2
a['content']=view2

# csv 형태로 저장하기
a.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
a.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 124 건
실제 최종 출력 건수 124
no 1 :  울산 소리체험관
울산 동구
울산광역시 동구 성끝길 103
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.25 ~ 별도 안내시까지)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기울산 슬도 대왕암에 위치한 소리체험관은 지상2층 654제곱미터 규모로 ''여음(소리의 잔향)의 풍경''을 컨셉으로 건축되었으며 울산 ''동구의 소리 9경''을 체험해볼 수 있는 체험전시관이다.


no 2 :  태화강 국가정원
울산 중구
울산광역시 중구 태화동
태화강은 울산의 중심을 가르며 흐르는 강으로, 화룡연을 굽이 돌아 학성을 지나면서 이 수삼산의 이름을 남기고 울산만에서 동해로 들어간다. 동서로 약 36㎢, 남북 28㎢ 의 유역은 그 대부분이 산악지대를 형성하나  강의 양쪽과 하류에는 기름진 평야가 펼쳐져 있으며, 오늘날에는 울산시민의 중요한 식수원이 되어주고 있다. 태화강의 심장부에 위치했던 태화들은 장기간동안 무관심으로 방치되어 있다가 태화강 국가정원 조성으로 다시 자연의 모습을 갖추고 시민들의 품으로 돌아왔다. 태화강 국가정원은 서울 여의도 공원 면적의 2.3배에 달하는 531천㎡로 물과 대나무, 유채ㆍ청보리를 비롯한 녹음이 함께 어우러진 전국 최대규모의 도심친수공간이다. 홍수 소통을 위하여 한때 사라질 위기에 처해졌던 십리대숲은 백만 시민의 단결된 힘으로 보전하게 되었고, 도시계획상 주거지역으로 결정되어 개발이 예정되어 있던 186천㎡의 토지를 다시 환원시켜 오늘의 태화강 국가정원을 조성하였다. 태화강 국가정원은 04년부터 10년 5월까지 총사업비 1.196억원(사유지매입 1.000 공사비 196)을 투입하여 실개천과 대나무 생태원,야외공연장,제방산책로 등 자연과 인간이 공존하는 친환경적인 생태공원으로 조성하였다. 


no 3 :  배내골(울산)
울산 울주군
울산광역시 울주군 상북면 배내로 1061
영남알프스는 산이 높고 골이 깊어 그 웅장한 산세만큼이나 많은 

no 13 :  울산대교 전망대
울산 동구
울산광역시 동구 봉수로 155-1
울산대교 전망대는 높이 63M(해발203M)로 화정산 정상에 위치해 있다. 전망대에 올라서면 2015년 5월 개통한 국내 최장이자 동양에서 3번째로 긴 단경간 현수교인 울산대교와 울산의 3대 산업인 석유화학, 자동차, 조선산업 단지 및 울산 7대 명산을 조망할 수 있다. 주간에 바라보는 울산의 전경과, 야간에 바라보는 공단과 도심의 야경은 색다른 즐거움을 제공한다. 전망대에는 동구 관광기념품 기프트샵과 카페 등을 운영하고 있어 볼거리·머물거리 모두 제공하고 있다.


no 14 :  외고산 옹기마을
울산 울주군
울산광역시 울주군 온양읍 외고산3길 36
숨 쉬는 그릇, 옹기가 옹기종기1957년 허덕만 옹이 이곳에 자리 잡으며 형성되기 시작한 옹기촌으로 마을 전체가 옹기로 어우러진 모습이 독특하고 멋스럽다. 외고산 옹기마을은 전국 옹기의 50% 이상을 생산하며, 옹기박물관에는 기네스북에 등재된 세계 최대 규모의 옹기가 전시되어 있다. 또한 매년 울산 옹기축제가 개최되어 다양한 체험과 행사를 관람할 수 있다.


no 15 :  울산체육공원
울산 남구
울산광역시 남구 문수로 44
총 275,000여 평의 체육공원은 자연적인 산림과 조화를 이루면서 조성되어 주변에 침엽수,육송 등 교목 9,221주와 영산홍, 백철쭉 등 관목 233,043본이 있다. 테마별로는 북측 진입 광장 중심부에 공원 현판석을 설치하였으며, 200여년 된 으뜸소나무 조형소나무, 마로니에 광장, 2002호반 산책로, 생태학습장, 장미원, 야외 공연장, 호반광장, 제1·2소공원, 벽천분수, 고사분수, 잔디광장 등이 조성되어 있다. 2002년 월드컵 경기와 관련하여 월드컵 대표팀 사인동판, 축구공, 신문스캐랩, 기념품 등이 문수경기장 1층 로비에 전시되어 있다. 문화행사관 및 월드컵 함성관 등에는 각 국사별 선수들이 입었던 축구복과 사인볼, 각국가별로 응원에 사용되었던 도구들이 잘 전시되어 있다. 또한 각 관에서는 유리벽을 통

no 31 :  태화루
울산 중구
울산광역시 중구 태화로 300
진주의 촉석루, 밀양의 영남루와 함께 영남을 대표하는 누각으로 울산의 전통성과 정체성을 고스란히 담은 대표적 유적이다. 643년(신라 선덕여왕 12년), 당나라에서 불법을 구하고 돌아온 자장대사가 울산에 도착하여 태화사를 세울 때 함께 건립했다. 태화강변 황룡연 절벽 위에 위치했으며 임진왜란 때 소실되었다가 ‘역사와 미래가 있는 태화강 만들기’ 프로젝트의 일환으로 복원되어 현재의 모습으로 재탄생했다. 태화강대공원, 십리대숲과 산책로로 이어지며 태화강이 한눈에 내려다보이는 멋진 조망으로 많은 사람들의 사랑을 받고 있다.


no 32 :  장생포 모노레일
울산 남구
울산광역시 남구 장생포고래로 246
장생포 모노레일은 울산 장생포 고래 관광단지에 위치하고 있고 고래박물관을 출발해 고래문화마을과 5D입체영상관을 거쳐 다시 박물관으로 돌아오는 총 1.3km 순환형 노선으로 지상 3~5m에서 장생포 앞바다, 고래문화마을, 울산대교, 울산공단 등을 한눈에 조망할 수 있다.봄에는 레일 밑으로 가득찬 벚꽃나무를 여름에는 푸른 나무의 생생함을 가을에는 단풍의 형형색색과 겨울에는 눈꽃 가득한 나무 숲을 감상할 수 있어 자연과 산업관광을 동시에 즐길수 있다.


no 33 :  강동 화암 주상절리
울산 북구
울산광역시 북구 화암길
주상절리는 단면이 육각형 내지 삼각형으로 된 긴 기둥 모양의 바위가 겹쳐져 있는 특이 지질의 하나이다. 이 곳 화암마을 해변 일대에 있는 주상절리는 신생대 제3기(약2,000만 년 전)에 분출한 현무암 용암(Lava)이 냉각하면서 열수축 작용으로 생성된 냉각절리이다. 그 생김새는 수평 또는 수직 방향으로 세워진 다량의 목재더미 모양을 하고 있는데, 길이는 수십미터에 이르며, 하나의 주상체 횡단면 긴쪽 대각선의 길이는 50㎝정도이다. 이 주상절리는 동해안 주상절리 가운데 용암 주상절리로는 가장 오래되어 학술적 가치가 높으며, 다양한 각도로 형성되어 있어 경관적 가치도 크다. 주상체 횡단면이

no 45 :  병영교회
울산 중구
울산광역시 중구 병영성길 89
1895년 1월 8일에 왕길지 선교사에 의해 병영교회가 시작되어 2014년 119년의 역사를 가졌다. 지역과 함께 하는 교회로써 도서실 개설(`95),교회 도서관 운영, 비디오 선교 활동을 하고 있다.


no 46 :  문수사(울주)
울산 울주군
울산광역시 울주군 청량읍 문수산길 514
문수사는 울주군 청량면 율리 산342번지에 자리잡은 절로 문수산의 이름을 딴 절이다. 이 문수산은 신라와 고려 때는 주로 영축산이라 불려오던 산이다. 천축국왕사성 부근에 있는 영축산의 이름이 불교를 따라 우리나라로 이동하여 온 것으로서 천축의 영축산은 부처님이 설법하시던 곳이며,  신선들이 살았고 독수리가 많이 있으므로 영축산 또는 축두, 축봉, 축대라고도 하였다는 것이다. 또 축령들이 산에 있으므로 이름하였고 혹은 산모양이 수리머리와 비슷하다는데서 온 이름이라고도 한다.  조선조에 와서는 문수산이라 하여 신증동국여지승람에 그렇게 나타난다. 또 이 산을 일명 청량산이 라고 하였는데 산 아래에 있는 청량면은 청량산에서 비롯된 이름인 것이다. 이 산을 문수산이라고 한것은 화엄경 제10법 운지로서 보살들이 살던 곳이어서 문수보살이 응현이 있었기 때문이었다.한편, 삼국유사 권 5 연회도명 문수점에서 문수대성과 변재 천녀가 응현하여, 무거설화에서도 문수보살이 나타난다. 무거설화를 보면  "경순왕은 백척간두에 선나라의 장래를 영취산의 문수대성의 계시를 받아 결정키로 결심하고 태자와 둘째 왕자를 거느리고 하곡현의 영취산을 찾아 길을 나섰다.  먼저 태화에 이르러 참배하고 또 길을 나섰는데 중도에서 길가에 한 동자승이 나타나더니 대왕께서 오실 줄 알고 산으로 인도하여 모시고자 왔다고 하였다.  왕은 다행이라 생각하고 크게 만족하여 길을 따랐다. 그러나 삼호 앞에서 태화강을 건너자 얼마가지 아니하여 동자승은 그 자취를 감추어 버렸다. 왕은 직감에 이 동자승이 문수 보살임을 느끼고 하늘이 이미 나를 저버리는구나 하여 크게 탄식하

no 58 :  화암추 등대
울산 동구
울산광역시 동구 화암등대길 50
방어진 서쪽 해안마을의 바다에 돌출된 검회색 바위 위에는 이상하리 만큼 하얗게 꽃문양을 연상시키는 무늬가 박혀 있다고 하여 “꽃바위(花岩)”라 불렀고, 주변 마을을 “꽃방마을”이라 불렀다. 아침 해가 떠오를 무렵 바닷물이 만조를 이루었을 때, 출렁이는 물결에 드리워진 꽃무늬는 더할 나위 없이 절경을 이루었고, 저녁 무렵 바닷물이 썰물로 빠져 나가면서 바다 속에 잠겨 있던 온갖 바위형상들이 만물상을 이루는 모습을 보고는 감탄하지 않는 이가 없었다고 한다. 이 아름다운 풍경을 시인묵객들은 “화암만조”(花岩晩潮)라 부르며 방어진 12경중 제1경으로 꼽기도 하였으나, 1989년 항만축조 및 매립사업으로 모두 사라져 버렸다.  1961년 이후 울산에 공업단지가 조성되면서부터 산업의 대동맥인 석유화학제품들이 쏟아졌고, 울산항에는 이를 실어 나르는 배들의 왕래가 잦아지게 되었다. 이에 따라 해양오염사고 방지 및 선박의 안전항해를 위한 항로표지 설치가 절실히 요구됐고, 울산항 관문에 위치한 꽃방마을 끝단에는 천년동안 안전하게 항해하기를 기원하는 거북이 모양의 등대가 세워졌다. 이후 조선산업의 발달로 항만 축조 및 매립이 이루어져 바다 쪽으로 등대를 옮겨야 할 필요성이 제기되었고, 낮아진 등고를 보완하고 울산항을 출입하는 선박들의 길잡이 역할을 원활히 수행하기 위해서는 높은 등대 설치가 요구되었다. 마침내 1994년 12월 동양최대 규모의 등대가 건립되어, 머나먼 여정을 헤쳐온 선박들을 안전하게 울산항으로 인도하게 되었다. 특히, 해상교통의 요충지인 이곳 화암추등대는 엘리베이터를 이용하여 전망대에 오르면 생동감 넘치는 울산항과 24시간 불빛을 내뿜는 산업단지의 역동성을 한 몸에 느낄 수 있으며, 등대 앞 방파제의 강태공들이 계절에 따라 색다른 손맛을 즐기는 모습을 볼 수 있다. * 최초 점등일 - 1983년 1월 28일 * 구조 - 백원형 콘크리트조(44m) * 등질 - 백홍호광 20초 1섬광(AlFlWR

...

no 109 :  열두개울
경기 연천군
경기도 연천군 청산면 청신로 345
열두개울은 기암절벽과 맑은 계곡이 손잡고 선경을 빚는다. 선녀바위, 무장소, 보안소,만장바위, 평바위, 도라소, 돌묵소, 봉바위, 쌍무소, 용수골소 등의 명소가 10리에 걸쳐 펼쳐진다.옛날, 다리가 없던 시절에 법수동에서 덕둔리로 가려면 열두차례나 개울(산내천)을 건너야 했다고 한다. 지금은 5개의 다리가 놓이고 길이 뚫려 교통이 편리해졌지만 `열두개울'' 이라는 이름은 여전히 남아 있다. 인근에는 1992년 12월에 온천지구로 지정받은 포천시 소재 신북온천이 있다.


no 110 :  연천향교
경기 연천군
경기도 연천군 연천읍 지혜로117번길 20
1407년(태종 7)에 현유(賢儒)의 위패를 봉안, 배향하고 지방민의 교육과 교화를 위하여 창건되었다. 1910년 이후 학술강습소로 사용되다가 1916년 보통학교로 개편되었으며, 1941년 일제의 강압으로 연천군 마전(麻田)의 삭녕학교(朔寧學校)로 합설되었다. 1950년 6·25전쟁 때 전소되었다가 1965년 이 지역 유림이 대성전 등을 중건하였다. 대성전에는 5성(五聖), 송조2현(宋朝二賢), 우리 나라 18현(十八賢)의 위패가 봉안되어 있다. 조선시대에는 국가로부터 토지와 전적·노비 등을 지급받아 교관 1명이 정원 30명의 교생을 가르쳤으나, 갑오개혁 이후 신학제 실시에 따라 교육적 기능은 없어지고 봄·가을에 석전을 봉행하며, 초하루·보름에 분향하고 있다. 현재 향교의 운영은 전교 1명과 장의 수명이 담당하고 있다.


no 111 :  연천 옥계마을
경기 연천군
경기도 연천군 군남면 군중로 36-10
황옥, 현옥, 백옥, 적옥, 창옥으로 불리는 오옥의 콩이 자라는 옥계마을은 임진강이 흐르는 곳이다. 특히 마을에서 아주 가까운 곳에 위치한 옥녀봉은 이곳에 있는 연못에 선녀가 내려와 놀다갔다고 이야기가 전해질 정도로 빼어난 경관과 깨끗한 물을 자랑하고 있다. 분단의 아픔을 싣고 흐르는 임진강은 주변에 많은 문화유적지가 있다.


no 1

no 126 :  나인포레스트 이화원
경기 가평군
경기도 가평군 가평읍 자라섬로 64
2015년 새롭게 오픈한 가평 이화원은 가평 자라 섬 내에 위치하여 있으며 총면적 1만여 평 부지, 나비생태관 330평으로 이루어져 있는 세계 최대의 나비공원이다. 살아있는 나비와 표본을 4계절 모두 관람할 수 있고 2,300여 평의 온실로 이루어져 있어 날씨에 관계없이 모두 편안하게 관람할 수 있다. 눈으로만 보는 것이 아니라 전문 해설사가 나비 및 식물에 대하여 설명을 해주어 관람객들에 거 더욱 많은 볼거리 및 유익한 정보를 제공하고 있다. 또한 자라 섬 인근 캠핑장 및 각종 페스티벌이 열리고 있다.


no 127 :  여주도자세상
경기 여주시
경기도 여주시 신륵사길 7
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.19 ~ 별도 안내시까지)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기* 보고,사고,만들고,쉬고 여주도자세상 *여주도자세상은 아름다운 아트샵, 한다발 리빙샵, 바람 브랜드샵, 손맛 갤러리샵 등 도자쇼핑몰로 구성되어 있으며, 4개의 상품관을 통해 전국 115개 요장, 6,300여 종의 다양한 도자기를 저렴한 가격에 판매하는 등 국내 도자 유통의 허브역할을 수행하고 있다. 도자 전문미술관인 반달미술관에서는 예술작품으로써 생활도자의 무한영역을 보여주는 도자 작품으로만 엄선하여 상설전이 열리고 있다. 이에 더하여 신륵사, 영녕릉, 명성황후 생가, 세종문 등 전통문화관광지가 많은 여주의 지역적 특성을 살려 대형 한옥건축물로 조성되어 여주를 대표하는 랜드마크가 되고 있다.


no 128 :  광주 곤지암도자공원
경기 광주시
경기도 광주시 곤지암읍 경충대로 727
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.19 ~ 별도 안내시까지)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기경기도 광주 삼리에 있는 광주 곤지암도자공원은 대부분의 부지가 구석기 유적지이며, 조선시대 왕실에 백자를 제조, 납품하는 

no 146 :  마장호수 출렁다리
경기 파주시
경기도 파주시 광탄면 기산로 313
깔끔하게 조성된 공원과 분수대를 감상하며 곳곳에 쉬어갈 수 있게 마련된 벤치, 야생화가 가득한 하늘계단, 마장호수 관광의 또 다른 재미의 수상레저(카누, 카약), 호수 둘레길이 낭만적인 곳으로 주말가족, 연인들의 나들이 장소로 제격이다.산과 호수를 끼고 있어 물빛과 낙조가 주변 군락목과 푸른산과 조화를 이루고 있는 모습이 아름다워, 답답한 일상에서 벗어나 자연의 품속에 파묻힌 듯한 편안한 휴식을 취할 수 있는 파주 대표적인 자연관광지이다.


no 147 :  김포함상공원
경기 김포시
경기도 김포시 대곶면 대명항1로 110-36
김포함상공원은 지난 62년간 바다를 지켜오다 2006년 12월 퇴역한 상륙함(LST)을 활용하여 조성한 수도권 최초의 안보의식 체험장 함상공원이다. 다양한 볼거리와 안보체험 등 독특한 경험을 할 수 있다. 문화관광해설사가 있어 해설사의 설명을 들으며 영상관, 선실재현공간, 한국전쟁 홍보관, 한주호 준위 추모관 등 전시관을 보다 재미있게 관람 할 수 있다. 상갑판을 지나 조타실, 전탐실을 올라가면 군인들이 생활하던 공간이 있고 이 공간을 통해 간접적으로 해군의 생활을 엿볼 수 있다. 운봉함의 가장 꼭대기인 함교에 올라서면 서해바다의 정취를 눈앞에서 느낄 수 있다.


no 148 :  반구정과 황희선생유적지
경기 파주시
경기도 파주시 문산읍 반구정로85번길 3
반구정은 고려말에서 세종조에 이르기까지 장기간 임금을 보필하고  89세로 장수하였던 청백리 ''황희정승이 말년에 관직을 사양하고  고향에 돌아와 갈매기를벗 삼아 지낸 곳으로 임진강 하류의 절경 지역에 세워진 정자이다. 바로 윗쪽에 앙지대(仰止臺)가 있다. 분단의 현실을 체감할 수 있는 철조망이 반구정 언덕과 임진강을 가로 막아 낙조가 들면 더없는 슬픔과 아름다움을 한꺼번에 느끼게 해주는 곳이다.허목(許穆)의 반구정기(伴鷗亭記)에 "조수때마다 백구가 강위로 몰려들어 모래사장 벌판에 가득하다" 고 하였

no 11 :  보덕사(영월)
강원 영월군
강원도 영월군 영월읍 보덕사길 34
* 단종의 극락왕생을 기원하는 사찰, 보덕사 *보덕사는 영월읍 북방 3km 떨어진 능동에 있는 고찰이다. 686년 의상조사가 창건하고 발본산 지덕사라 하였다고 전해지고 있으며, 일설에 의하면 714년 혜각선사(蕙覺禪師)가 창건하였다고도 한다. 1161년(고려 17대 인종 32년)에 설허선사와 원경국사가 극락보전, 사성전, 염불암, 고법당, 침운루 등을 증축하였다. 1456년 단종대왕이 노산군으로 강봉되어 이곳 영월로 유배되자 사찰명은 노릉사라 개칭하게 되었다. 1705년 숙종 31년에 한선선사와 천밀선사가 대종을 만들어 설치하게 되자 사찰명도 장릉 보덕사로 고쳤다. 그 후 1726년 (영조 2년) 6월에 태백산 보덕사라고 개칭하게 되었다. 지금은 평창군에 있는 오대산 월정사에 속한 제일 거찰로서, 1950년 한국전쟁 당시 건물의 대부분이 소실되었으나 지금은 거의 원형대로 복원되어 가고 있다. 영월 북방 발본산밑 바른편에 위치하고, 앞으로는 동을지산과 장릉이 한눈에 보인다.* 단종과 보덕사의 관계 *보덕사는 단종대왕이 노산군으로 강봉되어 이곳 영월로 유배되었던 곳이다. 단종의 극락왕생을 기원하는 보덕사 칠성각에는 태백산 산신령이 되었다는 단종의 혼령을 추모하기 위하여 단종의 영정을 모시고 있다. 극락보전 오른편에는 보덕사가 의상의 화엄도량으로 창건된 것임을 짐작케하는 부도1기가 있으며, 극락보전 안에 있는 삼존불은 목조불로 알려져 상당한 문화적 가치를 지니고 있다.


no 12 :  영월 청령포 관음송
강원 영월군
강원도 영월군 남면 광천리
* 단종 유배생활의 벗, 영월의 관음송 *한강 상류지역인 영월군 남면의 강 가운데 고립된 작은 섬의 소나무숲 중앙에 위치하고 있는 수령 600년의 천연기념물이다. 높이는 30m에 이르고, 가슴높이에서의 둘레가 5m나 된다. 밑에서 1.2m되는 높이에서 두 줄기로 갈라져 있는데, 하나는 위로 하나는 서쪽으로 약간 기울어져 있다. 관음송이라는 이름은

IndexError: list index out of range

In [3]:
#xPath로 게시글 수 가져온 것,, 먹힐까?

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt
from selenium.common.exceptions import NoSuchElementException
import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '관광지 울산부터'
# 파일 저장
f_dir = 'c:\\캡디_공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EA%B4%80%EA%B4%91%EC%A7%80&choiceTagId=3f36ca4b-6f45-45cb-9042-265c96a4868c')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
view2 = []      # 게시글 내용 컬럼


no = 1

for j in range(9,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()

    time.sleep(3)

    r_cnt= driver.find_element_by_xpath('''//*[@id="totalCnt"]''').text
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)
    
    cnttt = 150
    page_cnt = math.ceil(cnttt/10)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
#             if no > cnttt :
#                 break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    
            con_1 = content_list.find('p').get_text()
            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD = '0'
            except:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
            
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except IndexError:
                try:
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                except IndexError:
                    addr = '0'
                except NoSuchElementException:
                    addr ='0'
            except NoSuchElementException:
                try:
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                except NoSuchElementException:
                    addr = '0'
            except:
                addr = '0'
            
            
            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            print(con_1)
            f.write(str(no) + ';'+ str(title) + ';'+str(area_CD) +';'+ str(addr) +';'+ str(con_1) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)
            view2.append(con_1)
            
            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            
            if no > cnttt:
                break
        if i > page_cnt+1:
            break
        i += 1
        
        if no > cnttt:
            break
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
    i=0
    k=0
    no=1
        
a = pd.DataFrame()
a['no']=no2
a['title']=contents
a['area_cd']=area_CD2
a['addr']=addr2
a['content']=view2

# csv 형태로 저장하기
a.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
a.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 1333 건
실제 최종 출력 건수 150
no 1 :  애니메이션 박물관
강원 춘천시
강원도 춘천시 서면 박사로 854
* 만화? 보지만 말고 움직여봐 *애니메이션의 기원, 탄생, 제작과정과 세계 애니메이션의 역사를 살펴볼 수 있다. 입체 애니메이션 관람, 음향효과·더빙 체험, 인형 애니메이션 촬영 체험 등 예쁜 그림 안에서 재미있는 체험 놀이를 할 수 있다. 캐릭터와 함께 사진 찍을 곳도 많고 아이들에게 몰입도가 높은 박물관이다.


no 2 :  하슬라아트월드
강원 강릉시
강원도 강릉시 강동면 율곡로 1441
하슬라아트월드는 자연과 사람, 예술이 조화롭게 공존하는 종합예술공간이다. 3만 3천여 평의 조각공원은 ‘성성활엽길’, ‘소나무 정원’, ‘시간의 광장’, ''바다정원‘, ’하늘전망대‘, ’돌갤러리와 소똥미술관‘의 테마로 자연훼손을 최소화하기 위하여 비탈면과 산의 높이를 그대로 이용하여 손으로 직접 만들었으며 자연을 가장 아름답게 볼 수 있는 각도로 길을 내고 예술정원으로 꾸몄다. 하슬라아트월드는 국내외 작가들과 함께 하는 예술 작업 및 공연 전시 축제가 1년 내내 끊이지 않는 특별한 경험과 행복을 주는 공간이다. 실내미술관에는 현대 미술 200여 점이 전시된 ''현대 미술관''과 유럽 각국에서 수집한 마리오네트와 피노키오 작품들이 전시된 ''피노키오 & 마리오네트 미술관''이 있다. 하슬라 미술관은 기획전시, 초대전시, 상설전시가 진행 중이며 전시 스케줄에 따라 관람할 수 있다.1) 성성활엽길 예술작품과 300여 종의 야생화, 식물나무가 분포되어 있는 길로, 작가의 드로잉과 설치 작품으로 시작되는 로드 아트 또한 심심치 않게 발견하게 된다. 반사경이 바닥에 놓여 있어 고개를 들지 않아도 파란 하늘과 울창한 나무를 볼 수 있다.2) 소나무 정원우리나라의 대표적 수종이며 국민이 가장 선호하는 소나무가 테마인 정원이다. 돌과 부족한 흙과 바다바람에 자란 작은 소나무의 모습은 많은 나이에 비해 분재에서 느끼듯 애절하고 기묘한 분위기를

no 15 :  영모전
강원 영월군
강원도 영월군 영월읍 영모전길 148
* 단종을 기리는 곳, 영모전 *단종(재위 1452~1455)의 영정을 모신 곳으로, 영월의 주산인 매봉산 자락 남쪽 언덕에 있다. 이곳에 유배된 단종이 살해되자, 그를 추모하기 위해 군민의 성금을 모아서 1517년(중종 12) 영월부사 이용하(李龍夏)가 그의 영정과 위패를 모시려고 성황당이 있던 자리에 사당을 건립하였으며, 이용하의 후손 이계진이 개수하여 ‘영모전’이라는 현판을 걸었다. 또한 영정은 추익한(秋益漢)이 백마를 탄 단종에게 머루를 진상하는 모습으로 그려져 있다. 영모전은 앞면 3칸·옆면 2칸 규모이며, 지붕 옆면이 여덟 팔(八)자 모양인 팔작지붕집이다. 앞면 가운데 칸의 뒷부분에 초상을 모셔둔 방을 두고 그 양 측면과 뒷면은 벽을 설치하였으며, 앞면은 장지문을 달아 밝게 처리하였다. 그 외의 부분에는 마루를 놓았는데 앞면 3칸은 모두 문 없이 개방되어 있고, 뒷면과 옆면은 벽에 나무로 살을 대어 만든 살창을 설치하였다. * 단종이 산신령이 된 전설 * 추익한이 단종에게 산머루를 진상하려고 영월로 오던 중 연하리에 이르렀을 때, 곤룡포와 익선관 차림에 백마를 타고 동쪽을 향해 가는 단종을 만나게 되었다. 추익한은 황망히 읍하고 땅에 꿇어앉아 “대왕마마께서는 어디로 행차하시나이까？” 하고 여쭈니 단종이 “태백산으로 가는 길이오. 그것은 처소에 갖다두시오” 하고 말한 뒤 홀연히 사라졌다. 추익한은 기이하게 생각하고 급히 단종 처소에 가보니 단종은 이미 승하한 뒤였다. 추익한은 방금 전 만났던 것이 단종의 혼령이었음이 분명하다 생각하고 그의 뒤를 따라 목숨을 끊었다고 한다. 그래서 단종과 함께 태백산의 산신령이 되었다는 전설이 있다. 이 전설은 단종을 숭상하는 일종의 신앙으로 발전하여 영모전 외에 녹전 성황당, 정선각, 보덕사의 칠성각 등에서도 단종의 위패 또는 영정을 모시고 있다. 또한 그 뒤로도 군민들은 마치 단종이 살아 있는 것처럼 공경하여 해마다 단종문화제가 열리는 시기와 

no 29 :  법흥계곡
강원 영월군
강원도 영월군 수주면
영월 서강의 상류이며 남한강 발원지로 이루어진 수주면의 여러 계곡은 태고의 신비를 그대로 간직하고 있으며 맑은 물에만 서식하고 있는 가재, 도롱뇽, 쉬리, 황쏘가리, 자라등 희귀 토종 어종과 멸종위기 동물로 보호받고 있는 수달이 서식하고 있는 곳으로 반드시 보호되어야 하는 천혜의 자연보고이다. 사자산(1,120m)에서 발원해 법흥사 앞을 거쳐 주천강으로 흘러드는 계곡으로, 사암봉·사자산·백덕산(1,350m) 등 해발 1,000m 이상의 고봉에서 흐르는 맑은 물이 모인 탓에 옆새우·열목어 등 1급수에서만 볼 수 있는 물고기들이 자란다. 상류는 경사가 급해 물살이 빠르다.


no 30 :  소금강 (강원고생대 국가지질공원)
강원 정선군
강원도 정선군 화암면 몰운리
소금강은 계곡을 따라 기암절벽이 즐비해 마치 금강산을 닮아서 붙여진 이름이다. 소금강의 절벽이 멋진 이유는 무엇일까? 바로 소금강을 따라 분포하는 규암이나 사암은 대부분 석영으로 이루어져 있고 석영은 화학적 풍화를 받아도 토양(흙)이 잘 만들어지지 않는다. 즉, 토양으로 변해 흘러내리지 않기 때문에 이렇게 멋진 절벽을 이루고 있는 것이다. 암석이 지표면에 노출되면 큰 변화(풍화)가 생긴다. 작은 크기로 부서지는 것을 기계적 풍화, 비가 내리고 식물이 자라서 토양(흙)으로 변해가는 것이 화학적 풍화이다. 이 지역은 위에 있는 암석이 작게 부서져서 아래로 떨어져 내린 것으로 기계적 풍화를 받은 것이다. 암석 틈으로 물이 들어가서 얼면 부피가 커져 암석이 더 벌어지게 되고 커진 암석 틈으로 물이 다시 들어가서 얼면 더 큰 틈이 생긴다. 이러한 과정이 반복되면서 암석의 틈이 많이 벌어져서 아래로 떨어져 내린 것이다.<출처 : 강원고생대 국가지질공원>


no 31 :  삼악산(춘천)
강원 춘천시
강원도 춘천시 서면 경춘로 1401-25
서울에서 북쪽으로 80km,춘천에서 남서쪽으로 10km 떨어진 삼악산과 등선폭포(계곡)는 경춘 국도변에서 가깝기 때문

no 46 :  용추폭포
강원 동해시
강원도 동해시 삼화로 538
용추폭포는 청옥산에서부터 갈미봉에 이르는 능선 밑에서 발원하여 바른골로 흘러내린 물이 떨어지는 폭포이다. 폭포는 상, 중, 하 셋으로 된 3단 폭포로서 하탕폭포 밑은 둘레가 약 30미터나 되는 깊은 소이다. 용추폭포에서 내려오는 물은 박달계곡의 물이 이곳에서 만난 것이 마치 자연의 음양의 섭리와 순리를 나타내는 듯하다. (출처 : 강원도청)


no 47 :  잣나무 연리지
강원 태백시
강원도 태백시 소도동 산 33-1
태백시 소도동 산33-1번지에 위치한 잣나무 연리지는 약 200년으로 추정되며 작은 나무는 둘레가 약 1.5m, 큰 나무는 약 2m이며 높이는 20m에 달한다. 소나무와 달리 이곳 잣나무 연리지는 자라서 연리목이 되었다. 특히 이 연리목은 수종이 서로 다른 느티나무와 개서어나무의 몸통전체가 결합되어 있어 더욱 상서로운 나무로 알려져있다. 이 나무 앞에서 서로 손을 꼭 잡고 기도하면 부부간의 애정이 더욱 두터워지고 남녀간의 사랑이 이루어지며 소원성취 한다고 전해지나. 문헌상 삼국사기의 신라 내물왕 7년 시조묘의 나무와 고구려 양원왕 2년 서울의 배나무가 연리지가 된 기록과 고려사의 광종 24년 , 성종 6년에 연리지의 출련을 기록했을 정도로 상서로운 나무라 전해진다. 연리목과 연리지는 부부간의 금술이나 남녀간의 애정이 깊음을 비유한다. 특히 이 연리목은 수종이 서로 다른 느티나무와 개서어나무의 몸통전체가 결합되어 있어 더욱 상서로운 나무로 알려져있다. 이 나무 앞에서 서로 손을 꼭 잡고 기도하면 부부간의 애정이 더욱 두터워지고 남녀간의 사랑이 이루어지며 소원성취 한다고 전해지나.(출처 : 강원도청)


no 48 :  필리핀군전투전적비
강원 양구군
0
철원-평강-김화를 잇는 철의 삼각지대에서 치열한 전투가 있었다. 1952년 05.18 ~ 21일과 1952.06.18. ~ 21일 두 차례에 걸쳐 중공군과 의용군을 필리핀군 제 19, 20 전투단이 힘껏 싸웠다. 제 1차 전투 3일 

no 65 :  평창 지동봉가옥
강원 평창군
강원도 평창군 평창읍 천동길 160
평창읍 천동리에 있는 일제강점기의 가옥이다. 강원문화재자료 제 74호로 지정돼었으며, 언제 건립되었는지는 정확하게 알려지지 않았으나 기와에 남아있는 명문으로 볼 때 일제강점기인 1941년에 세워진 것으로 추정되는 가옥으로 지동봉이 소유하였다. 대지 52평에 안채와 행랑채, 별채로 이루어진 가옥으로 지봉동이란 사람의 증조부가 분가하여 집을 지은 뒤 지금까지 4대가 걸쳐서 살고 있는 역사가 깊은 가옥이다.안채와 사랑채, 별채로 이루어졌지만 지어졌던 당시의 상태 그대로를 유지하고 있는 것은 안채뿐이다. (출처 : 강원도청)


no 66 :  원당계곡
강원 평창군
강원도 평창군 평창읍 원당리
강원도 평창군 평창읍 원당리에 있는 길이 6km의 계곡이다. 이곳은 평창강 유원지와 뇌운계곡에 가려 그다지 많이 알려진 곳은 아니다. 따라서 인적이 드문 이곳은 그대로 보존된 원시계곡의 아름다움을 고스란히 간직하고 있다. 또한 물이 깊이 않고, 물살이 거세지 않아 우리나라 토종 어종인 쉬리를 비롯하여 다양한 민물고기들의 서식지가 된다. 계곡 끝자락 산 중턱에는 ‘백덕사’라는 절이 있다. 새벽예불과 더불어 아름다운 일몰과 출몰을 볼 수 있는 해맞이 장소로도 사랑받고 있다. 백덕사 주변에는 야영장이 많이 있어 도시를 벗어나 자연 속에서 조용한 여행을 즐기려는 캠핑족들에게 더할 나위 없이 적합한 장소이다. 주면 명소로 금당산, 율곡선생사당, 뇌운계곡, 봉황대, 금당계곡, 무이골 등이 있다.(출처 : 강원도청)


no 67 :  고생대 화석산지
강원 태백시
강원도 태백시 혈동 사내골 노두
규모 475㎡의 고생대 화석산지는 장성동의 명소이다. 천연기념물 제 416호로 지정된 문화재이며, 고생대의 다양한 종류의 화석들이 출토되었다. 고생대에 한반도가 적도 부근에 위치하고 있다는 자연적 역사를 알려주고 있어 그 가치가 매우 크다. 이를 토대로 태백고생대자연사박물관을 건립하였다. 장성의 고생대 화석산지와 태백 

...

no 136 :  화동서원
전북 김제시
전라북도 김제시 장화동
정씨 효자 3형제를 기리는 서원이다. 김제시 장화동에 있다. 화동서원에 배향된 정씨 3형제는 정윤교, 정윤준, 정윤검이다. 이들은 16년 동안 상복을 입고 전국을 누비며 무고로 사망한 아버지의 원수를 갚았다. 그리고 그 후 동헌에 자진출두해 벌 받기를 자청했다. 그 효성에 감복해 고을 사또가 3형제 중 첫째만 옥에 가두었다. 그러자 나머지 두 동생이 함께 벌을 달라 무릎꿇어 호소했다. 저간의 사정을 들은 인근 선비들이 탄원해 첫째가 풀려날 수 있었는데, 가뭄을 앓던 마을에 단비가 내려 해갈되었다고 전한다. 그 후 3형제는 벼슬을 마다하고 고향에서후진을 양성하는 데 온 힘을 쏟았다고 한다. 서원은 인근 지방의 유림들이 이들 3형제의 효심과 우애를 본받고자 1924년에 세웠다. 


no 137 :  육송사
전북 김제시
전라북도 김제시 금산면 삼봉길 203-7
언양김씨를 배향하고 있는 사당이다. 김제시 금산면 삼봉리에 있다. 본래 육송사에는 송정기, 송정구, 송정모 등 효자로 이름난 송씨 삼형제가 배향되었으나 금구면 불로마을 낙양사로 옮겨갔고, 그 자리에 김관, 김승속, 김승서 등 언양김씨의 뛰어난 선비들을 모시고 있다. 김관은 전라도관찰사와 황해도관찰사 등을 두루 역임하고 찬성 벼슬에 이른 인물이고, 김승속은 우애와 효심이 깊었던 인물로 어려서 사마시에 급제했다. 김승서는 예번시참봉의 자리에서 뭇 사림의 존경을 받았다. 육송사는 숙종 때 창건되어 고종 때 잠시 철폐되었다가 1898년(광무 2) 그 후손들에 의해 복원됐다. 정면 3칸, 측면 2칸으로 이루어진 맞배지붕 형태의 사당과 정면 4칸, 측면 2칸으로 이루어진 팔작지붕 형태의 재실 두 채가 있다.


no 138 :  삼불암
전북 김제시
전라북도 김제시 죽산면 서포4길 443
삼불암은 자연석으로 이루어진 세 개의 불상을 봉안한 한국불교 태고종 소속 사찰이다. 김제시 죽산면 서포리 불당마을에 자리하고 있다. 삼불암의 세 불상은 신묘한 전설과 연결되

전체 검색 결과 건수 : 1024 건
실제 최종 출력 건수 150
no 1 :  율포해수녹차센터
전남 보성군
전라남도 보성군 회천면 우암길 21
※ 코로나바이러스감염증-19 공지사항※ 내용 :임시휴관 (2020.08.22 ~ 2020.10.11)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기지난 1998년 개장이후 보성군이 직영하고 있는 율포해수녹차센터는 전국의 수많은 관광객과 이용객들로부터 사랑을 받고 있는데, 특히 지하 120m에서 지하 바닷물을 끌어올린 암반해수와 차재배 주산지인 보성군내 다원에서 생산된 차잎을 우려낸 녹수를 이용하여건강목욕을 즐길 수 있고 백사청송의 풍광과 함께 올망졸망한 남해안의 정취를 함께 조망할 수 있다.[출처:보성 문화관광]


no 2 :  한국차박물관
전남 보성군
전라남도 보성군 보성읍 녹차로 775
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.22 ~ 2020.10.11)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기한국 차, 제대로 즐겨보자차 주전자를 닮은 이 박물관은 대한다원 아래 위치하였다. 차 문화실, 차 역사실, 체험 공간으로 구성되어 있고, 다례 교육, 차 만들기 체험 등을 해 볼 수 있다.


no 3 :  고하도 전망대 및 해안데크
전남 목포시
전라남도 목포시 고하도안길 234
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.22 ~ 2020.09.27)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기* 고하도 전망대과거 이충무공이 명량대첩 승리 후 106일 동안 전열을 가다듬었던 고하도에 이충무공의 얼을 기리고자 13척의 "판옥선 모형"을 격자형으로 쌓아올린 건물이다.* 고하도 해안데크총 연장 1,080m로 전체구간이 해상에 위치하여, 고하도 해안 자연절경인 해식애와 목포 해안을 동시에 조망 할 수 있으며, 시원하게 펼쳐진 푸른바다와 높이 해발 228m 목포를 대표하는 유달산, 다도해의 관문 낭만항구 목포항 그리고 저녁노을이 아름다운 야

no 16 :  보성강
전남 곡성군
전라남도 곡성군 목사동면
보성강은 국가하천으로 곡성군의 구간은 목사동면 신기리에서 오곡면 압록리까지 섬진강과 합류하는 약18㎞로서 곡성에서는 대황강이라고도 한다. 옛부터 보성강에 8대어전(일명 : 쏘)이 있어 은어, 메기, 쏘가리, 참붕어, 참게, 잉어 등 담수어가 많이 서식하여 지역주민들은 봄철이면 투망으로 물고기를 잡아 매운탕을 끓여 먹고 노는 천렵이 성행하였으며 여름철에는 횃불을 들고 물고기를 잡는 횃불놀이가 아름답다하여 곡성의 팔경 중 하나인 대황어화(大荒漁火)로도 유명하다. 특히 물이 맑고 깨끗하여 은어, 조개 등 어패류가 많아 이를 잡아 생계를 유지하는 농가도 있었는데 주암댐 건설로 수심이 낮아지면서 어패류의 수가 점차 감소하는 등 생태계가 변화하고 있어  보성강 본래의 모습을 되찾고자 강변공원조성, 자연친화형 소하천 정비사업 등을 추진하여 점차적으로 본래의 모습을 되찾아 가고 있고, 강태공들의 발길도 끊임없이 이어지고 있다. 주변에는 곡성의 옛 현터인 욕천현의 동헌터, 무형문화재인 돌실(가는삼베)나이와 낙죽장도, 반구정 유원지, 태안사, 신숭겸장군의 유적지 등이 있다. 


no 17 :  생일도
전남 완도군
전라남도 완도군 생일면
생일도(生日島)는 해발 483m의 백운산과 금곡해수욕장을 낀 동백숲이 있으며, 남쪽에는 용출리 갯돌밭이 장관을 이루는 섬이다. 생일도란 지명은 처음에는 산일도(山日島), 산이도(山伊島)라 불리다가 주민들의 본성이 착하고 어질어 갓 태어난 아기와 같다하여 날생(生)과 날 일(日)자를 붙여 생일도라 부르게 되었다고 한다. 또 하나의 유래는 예로부터 험한 바다에서 조난 사고와 해적들 횡포가 심해 이름을 새로 짓고 새로 태어나라는 뜻에서 생일도라고 불렀다는 설이 있다.백운산 서쪽에 위치한 금곡해수욕장은 최근까지 전혀 알려지지 않았던 곳이나, 면소재지인 유촌리와 해수욕장간 도로가 개통됨에 따라 빛을 보기 시작하였다. 폭 100m, 길이 1.2km로 주변에 후박과 잣밤, 동백나무 등 상록수림이 

no 32 :  안도해변
전남 여수시
전라남도 여수시 남면
금오도, 소리도, 횡간도 등과 함께 금오열도를 이루는 안도의 동북쪽 해안에 위치한 해수욕장이다. 금오열도의 다른 섬들과 함께 다도해해상 국립공원에 속하는 안도는 작은 섬이면서도, 바다가 깨끗하고 풍광과 경관이 수려해서 여름철이면 많은 피서객들이 찾는다. 섬 곳곳에는 조용하고 아담한 해안이 펼쳐져 있어서 가족과 함께 피서를 즐기려는 사람들에게 더없이 좋은 휴양지이다. 또한, 섬 주변의 바다는 연중 낚시꾼들의 발길이 끊이지 않는 바다낚시터로도 유명하다.


no 33 :  고산 윤선도 유적지
전남 해남군
전라남도 해남군 해남읍 녹우당길 135
조선조의 문신(文臣)이자, 국문학의 비조로 일컬어지는 고산 윤선도(1587-1671) 선생의 유적지로 사랑채인 녹우당이 있다. 600년 전통을 이어온  해남윤씨 어초은파의 종가 고택인 녹우당은 덕음산을 뒤로 우리나라 최고의 명당자리 중의 하나로도 알려진 곳이다. 현재 고산유적지에는 오랜 역사와 전통을 말해주는 은행나무와 비자림, 사랑채인 녹우당, 안채, 행랑채, 헛간, 안사당, 고산사당, 어초은사당 ,추원당, 고산유물전시관등이 있어 조선시대 양반가 중 가장 많은 유물 (5,000여 점)을  보관해온 집안으로 잘 알려진 곳이다.


no 34 :  백수해안도로
전남 영광군
전라남도 영광군 백수읍 대신리
국도 77호선, 군도 14호선인 백수해안도로는 전남의 명 드라이브 코스중 하나다. 도로를 달리다 보면 해안절벽 사이로 솟아 있는 멋진 바위들과 여기저기 자리한 암초들이 다이나믹한 풍경의 변화를 연출한다. 영광군 백수읍 길용리에서 백암리 석구미 마을까지 16.8km에 달하는 해안도로로, 칠산바다의 아름다운 정경을 한눈에 볼 수 있으며, 특히, 일몰때 서해낙조는 동해의 일산 일출과 함께 쌍벽을 이루는 아름다운 광경으로 관광객들의 마음을 사로잡아 사계절 계속해서 관광객들의 발길이 이어지고 있다.


no 35 :  불갑사(영광)
전남 영광군
전라남도 영광군 불갑면 불갑사로 4

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"link text","selector":"6"}
  (Session info: chrome=85.0.4183.121)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.18363 x86_64)


In [4]:
#xPath로 게시글 수 가져온 것,, 먹힐까?

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt
from selenium.common.exceptions import NoSuchElementException
import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '관광지 제주만'
# 파일 저장
f_dir = 'c:\\캡디_공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EA%B4%80%EA%B4%91%EC%A7%80&choiceTagId=3f36ca4b-6f45-45cb-9042-265c96a4868c')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
view2 = []      # 게시글 내용 컬럼


no = 1

for j in range(16,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()

    time.sleep(3)

    r_cnt= driver.find_element_by_xpath('''//*[@id="totalCnt"]''').text
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)
    
    cnttt = 150
    page_cnt = math.ceil(cnttt/10)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
#             if no > cnttt :
#                 break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    
            con_1 = content_list.find('p').get_text()
            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD = '0'
            except:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
            
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except IndexError:
                try:
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                except IndexError:
                    addr = '0'
                except NoSuchElementException:
                    addr ='0'
            except NoSuchElementException:
                try:
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                except NoSuchElementException:
                    addr = '0'
            except:
                addr = '0'
            
            
            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            print(con_1)
            f.write(str(no) + ';'+ str(title) + ';'+str(area_CD) +';'+ str(addr) +';'+ str(con_1) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)
            view2.append(con_1)
            
            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            
            if no > cnttt:
                break
        if i > page_cnt+1:
            break
        i += 1
        
        if no > cnttt:
            break
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
    i=0
    k=0
    no=1
        
a = pd.DataFrame()
a['no']=no2
a['title']=contents
a['area_cd']=area_CD2
a['addr']=addr2
a['content']=view2

# csv 형태로 저장하기
a.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
a.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 344 건
실제 최종 출력 건수 150
no 1 :  한림공원
제주 제주시
제주특별자치도 제주시 한림읍 한림로 300
한라산(1,950.1m)의 서쪽, 한림읍에 위치한 비경지대이다.  제주시에서 일주도로 해안을 따라 서쪽으로 33㎞지점에 위치한 한림공원은 협재, 금릉 해수욕장의 비경과 아름다운 비양도의 경관을 마주하고  있는  제주도내 최고의 관광명소로 손꼽히고 있다. 10만여 평의 대지에 하늘로 우뚝 뻗은 야자수군락과 울창한 송림으로 둘러싸인 한림공원은 다양한 종류의 꽃과 나무가 사계절  아름다운 경관을 연출하며,  협재 쌍용동굴과 아열대  식물원 그리고 제주석, 분재원, 재암민속마을, 수석전시관, 사파리조류원 등 다양한 볼거리를 함께 갖추고 있다.  또한, 2002년에는 1,000여 평 규모의 연못을 새로 조성하여 선보이는 등 계속해서 다양한  볼거리를 만들어 나가고 있다.  한림공원은 1971년 초 한림읍 협재리 바닷가 일원의 불모의 모래밭을 매입하여 수천 트럭  분량의 흙을 실어와 객토작업을 하고 아열대 수목의 종자를 파종하여 육묘함으로써 탄생하게 되었다고 한다. 16개의 식물원을 보유한 식물나라이다. * 협재.쌍용동굴250만 년 전 한라산 일대의 화산이 폭발 하면서 용암이 흘러내려 형성된 검은 색의 용암동굴이 스며드는 석회수로 인하여 황금빛 석회 동굴로 변해가는 신비스러운 모습을 볼    수 있는 세계 유일의 2차원 복합동굴로 국가지정문화재 천연기념물 제236호로 지정되어 보호받고 있으며, 석회동굴에서만 볼 수 있는 석순과 종유석들이 자라고 있는 용암동굴로서 학술적 가치를 크게 인정받고 있다. 특히, 쌍용동굴은 그 형태가 마치 두 마리의 용이 빠져 나온 모양을 하고  있어서 쌍용동굴이라 부르고 있으며, 협재.쌍용동굴 이외에도 황금굴, 소천굴 등 20여 개의 동굴이     하나의 시스템으로 형성되어 있는데, 그 길이는 1만 7천m에 달하고 있어서 세계 최장의    용암동굴시스템으로 확인되었다.* 아열대 식물원 2천여 종의 진귀하

no 11 :  종달리 해안도로
제주 제주시
제주특별자치도 제주시 구좌읍 종달리
종달해안도로는 광치기해변에서 시작하는 해맞이해안도로의 일부로 하도해변까지 이르는 길로 약 11.2km 이다. 동쪽에 있는 해안도로라서 해맞이라는 이름이 붙었고, 해가 뜰 때도 물론 장관이지만 해가 질 무렵에도 아름다운 하늘과 바다의 색을 뽐낸다. 전동기로는 약 45분, 빠른 걸음으로는 약 2시간 13분, 보통 걸음으로는 3시간 43분 정도가 소요된다. 종달리해안에는 수국이 가득 심어져 있어 5월 말부터 6월 내내 활짝 핀 수국이 가득한 길을 지날 수 있다. 길이 구불구불하여 우도와 성산일출봉이 번갈아 보이며,  종달리 전망대에 올라 주변 풍경을 한눈에 담아보는 것도 좋다.마지막 지점인 하도해변은 조용한 분위기에 에메랄드 빛을 띠고 있으며, 건너편의 철새 도래지를 바라보며 여유를 느끼기에 제격이다.


no 12 :  제주탐나라공화국
제주 제주시
제주특별자치도 제주시 한림읍 한창로 897
탐나라공화국은 현장의 돌과 땀방울로 빚은 제주자연 상상나라이다. 또한 탐나라공화국은 여행자가 함께 가꾸어 간다. 여권 소지자는 국민이고 국민은 탐나라의 주인이다. 자신의 이름으로 나무를 심고 꽃밭을 일구는 땅이다. 상상으로 상상을 가꾸는 대한민국 상상나라이다.


no 13 :  혼인지
제주 서귀포시
제주특별자치도 서귀포시 성산읍 혼인지로 39-22
삼성혈에서 태어난 탐라의 시조 고.양.부 3신이 수렵생활을 하며 지내다가 동쪽나라(벽랑국)에서 온 세 공주와 합동혼례를 올렸다는 조그마한 연못이다.얕고 작은 연못에 불과하지만,  삼신인이 이곳에서 혼례를 올림으로써 비로소 제주민이 늘어나고 농사가 시작되었다 한다. 혼인지 바로 옆에는 삼신인이 혼례를 올린 후 신방을 차렸던 조그만 굴이 있는데, 그 굴이 세 갈래로 되어 있어 순전히 전설만은 아닌 듯하다.이곳은 삼성혈과 함께 제주도 시조의 자취를 더듬어 보기 좋은 곳으로 여름철에 피어나는 붉은 연꽃은 노을처럼 곱기만 하다.


no 14 :  제주 노루생태

no 29 :  생각하는 정원
제주 제주시
제주특별자치도 제주시 한경면 녹차분재로 675
수백여점의 분재가 오름과 물을 모티브로 한 자연정원에 자리하고 있다. 2007년 분재예술원 개원 15주년을 맞이하여 새롭게 태어나고, 세계적인 정원으로 나아가고자 생각하는 정원으로 정원의 이름을 정식으로 바꾸었다. 생각하는 정원의 역사는 1963년 성범영 원장이 제주에 첫발을 디디고 돌투성이 불모의 땅을 일구기 시작하는 것으로부터이다. 황량한 불모지에 꽃피운 한 농부의 혼불이라는 점에서 세계 언론의 주목을 받고 있으며, 실제 세계명사들이 가장 많이 다녀간 명소이다. 중국 장쩌민 국가주석의 방문과 후진타오 현 주석, 북한 김용순 노동당 비서, 일본 나까소네 총리대신 등 외교사절과 국내외명사들의 방문은 일일이 열거할 수도 없을 정도이다. 현재 이곳에는 400여점의 분재가 전시되어 있으며, 제주 화산석으로 쌓은 돌담과 돌탑, 정원 내에 시원하게 쏟아지는 제주 최대의 인공폭포, 또 하나의 폭포에서 쏟아져 내리는 물줄기는 커다란 잉어들이 한가로이 노니는 연못 등의 공원 시설이 잘 가꾸어져 있다.


no 30 :  함덕 서우봉 해변 (함덕해수욕장)
제주 제주시
제주특별자치도 제주시 조천읍 조함해안로
* 매혹적인 비취빛 산호바다, ‘카약’타고 노닐다 *제주 시외버스터미널에서 동쪽 약 14km 지점에 있으며, 일주도로에 위치해 있어 교통이 매우 편리하다. 수심이 아주 얕고 경사가 완만하여 아이들과 함께 해수욕을 하기에 적합하며, 바다가 얕아지면서 형성된 하얀 패사층은 마치 산호 바다와 같이 맑은 바닷물을 더욱더 빛나게 만든다. 하얗게 눈부신 모래밭과 맑은 에메랄드빛 바다, 그리고 검은 현무암 위에 가로 놓여진 아치형 구름다리, 빨간 등대 등이 한데 어울려 아름다운 풍광을 연출한다. 함덕해수욕장은 특이하게도 백사장 한가운데가 튀어 나와 마치 하트의 형상을 띄고 있기 때문에 동서의 어느 쪽에서 바람이 불어도 반대쪽 바다는 잔잔하기 때문에 항시 카약을 즐기기에 안성맞춤인 국내의 유일한 해수

no 45 :  용머리해안
제주 서귀포시
제주특별자치도 서귀포시 안덕면 산방로 218-10
산방산 앞자락에 있는  바닷가로 산방산 휴게소에서 10여분 걸어내려가면 수려한 해안절경의 용머리 해안과 마주치게 된다.이 곳은 수천만년 동안 쌓이고 쌓여 이루어진 사암층 중 하나이다. 해안 절벽을  모진 파도가 때려서 만들어 놓은  해안 절경이다. 작은 방처럼 움푹 들어간 굴방이나 드넓은 암벽의 침식 지대가 펼쳐져 장관을 이루고 있는 곳으로 얼마 전부터 제주의 새롭게 소개되는 관광코스 중의 하나이다. 용머리라는 이름은 언덕의 모양이 용이 머리를 들고 바다로 들어가는 모습을 닮았다하여 붙여졌다. 전설에 의하면 용머리가 왕이 날 휼륭한 형세임을 안 진시황이 호종단을 보내어 용의 꼬리부분과 잔등 부분을 칼로 끊어 버렸는데 이때 피가 흘러내리고 산방산은 괴로운 울음을 며칠째 계속했다고 한다.용머리로 들어가는 입구에는 하멜기념비가 서 있다. 용머리를 관광하는데는 한시간 정도 소요된다. 용머리는 CF와 영화의 촬영 장소로 이용되었다. 길이 30~50ｍ의 절벽이 마치 물결 치듯 굽어져 있다. 연인들의 산책코스로도 좋으며 차를 세워두고 한바퀴 돌아보는데 30분 정도 소요된다.  파도가 하얗게 부서지는 바닷가 바로 옆으로 길이 나있다.해녀들이 직접 잡아왔다는 해삼을 좌판에 내놓고 판다.해안도로를 따라 대정쪽으로 옮기면 송악산이다. 높이는 겨우 104ｍ이지만 바다와 직벽을 이루고 있는 명소이다. 초원에는 조랑말들이 한가롭게 노닐고 있고, 바다에는 형제섬이 그림같이 떠 있다. 절벽아래에는 일제때 일본인들이 연합군의 공격을 피해 팠다는 굴이 보인다.


no 46 :  방주교회
제주 서귀포시
제주특별자치도 서귀포시 안덕면 산록남로762번길 113
세계적인 건축가 이타미 준이 노아의 방주를 모티브로 설계한 교회건축물로, 아름다운 건축물로 유명하다. 인공 수조를 조성해 건물이 물위에 떠있는 듯한 느낌을 주며, 그 모습이 푸른 잔디밭, 파란 하늘과 잘 어울려 건축물과 자연의 조화로움을 뽐낸다. 사

no 65 :  쇠소깍
제주 서귀포시
제주특별자치도 서귀포시 쇠소깍로 128
제주도 서귀포시 하효동에 있는 쇠소깍은 원래는 소가 누워있는 형태라 하여 쇠둔이라는 지명이었는데, 효돈천을 흐르는 담수와 해수가 만나 깊은 웅덩이를 만들고 있어 ‘쇠소깍’이라고 붙여졌다. 쇠는 소, 소는 웅덩이, 깍은 끝이라는 의미를 담고 있다. 쇠소는 용암이 흘러내리면서 굳어져 형성된 계곡 같은 골짜기로 이름 만큼이나 재미나고 독특한 지형을 만들고 있다. 쇠소깍은 서귀포칠십리에 숨은 비경 중 하나로 깊은 수심과 용암으로 이루어진 기암괴석과 소나무숲이 조화를 이루면서 아름다운 풍광을 연출한다.  쇠소깍이 위치한 하효동은 한라산 남쪽 앞자락에 자리 잡고 있어 감귤의 주산지로 유명하여 마을 곳곳에 향긋한 감귤 냄새가 일품이다. 주변에는 주상절리대와 중문해수욕장, 천제연 폭포 등 유명한 관광지가 많아 함께 둘러보기에 좋다.


no 66 :  엉덩물계곡
제주 서귀포시
제주특별자치도 서귀포시 중문관광로 192
중문관광단지의 롯데호텔 동편으로 자리한 계곡이다. 큰 바위가 많고 지형이 험준하여 물을 찾는 짐승들조차 접근은 못하고 엉덩이를 들이밀고 볼일만 보고 돌아갔다고 해서 엉덩물계곡이라는 이름이 붙었다고 한다. 봄이면 계곡 경사면을 따라 유채꽃이 만발하여 장관이다. 입장료를 받는 다른 유채꽃 단지와는 달리 무료로 입장할 수 있으며, 평지보다 유채꽃 풍경이 입체적이다.올레 8코스에 포함되어 있으며, 중문달빛걷기공원으로도 불린다.<출처: 비짓제주>


no 67 :  선흘곶자왈 (제주도 국가지질공원)
제주 제주시
제주특별자치도 제주시 조천읍
선흘곶자왈은 동백동산으로 대표되는 울창한 산림지대이며 동백동산 습지는 2011년 람사르(Ramsar) 습지로 지정되기도 하였다. 동백동산 습지는 하천이나 호소 유역에 형성된 습지와 달리 곶자왈 지역에 형성된 내륙습지로서 소규모 연못 및 우기시 습지로 변하는 건습지 등이 곳곳에 분포하고 있다. 선흘곶자왈은 거문오름에서부터 북오름을 지나 선흘1리까지 이어진 곶자왈

...

no 93 :  한라산 영실
제주 서귀포시
제주특별자치도 서귀포시 영실로 246
제주 한라산은 자연휴식년제를 실시 중인 돈내코 코스를 제외한 다섯 개의 탐방코스가 있다. 관음사, 어리목, 영실, 성판악을 기점으로 하는 코스와 어리목 부근의 기생화산인 어승생악 코스가 있다. 한라산 최고의 가을 단풍코스로는 단연 영실코스를 들 수 있다. 한라산 서측 99번 국도를 따라가면 어리목 입구를 지나 1100고지 휴게소를 넘어서 약 4km쯤 내려가면 영실입구 삼거리에 도착한다. 삼거리에서 매표소까지는 약 3km 거리. 매표소에서 영실휴게소까지는 다시 30~40분 정도 걸어 올라가야 한다. 영실 탐방코스(영실∼윗세오름)는 3.7km로 대략 1시간 40분 정도 소요된다. 영실휴게소를 출발하여 1km 남짓 비교적 완만한 길로 힘들이지 않고, 주변 수목과 영실 계곡 단풍을 즐기며 쉬엄쉬엄 오를 수 있다.본격적으로 들쭉날쭉한 기암괴석이 시야에 들어오기 시작하는 오르막길로 접어들면 가파른 산길이 능선을 따라 이어진다. 등산로 아래로 깊게 팬 영실 계곡과 주변을 둘러친 웅장한 거벽은 아마도 이곳이 과거의 거대한 화산분출구가 아니었나 짐작케 한다. 병풍바위를 지나면서부터는 다시금 평탄한 산길이 나타나고 휘파람을 불며 거닐만큼 편안한 등산로가 윗세오름휴게소까지 이어진다. 한라산 영실의 첫번째 매력은 영주십경(瀛州十景)의 하나인 영실기암을 들수 있다. 해발 1,400∼1,600m 지점의 거대한 계곡 우측에 천태만상의 기암 괴석들이 즐비하게 하늘을 찌를 듯 솟아 있다. 옛부터 ''오백나한'', 또는 ''오백장군'' 이라 불리우기도 하는 영실기암은 깊어가는 한라산 단풍을 배경으로 가을 한라산의 진수를 펼쳐보인다. 병풍 바위에서 불어오는 산바람에 잠시 숨을 돌리고 땀을 식힌 다음, 내친 걸음에 윗세오름까지, 마치 광활한 활주로에 멀리 떡하니 버티고 있는 듯한 한라산 화구벽은 영실 가을산행의 또 다른 매력으로 다가온다. 여유가 있다면 윗세오름에서 북쪽 하산길로 접어 들어 봄철 철쭉으로 유명한 만세

no 104 :  마라도 가파도 정기여객선
제주 서귀포시
제주특별자치도 서귀포시 대정읍 최남단해안로 120
국토최남단 제주도 마라도 가파도 정기여객선 아름다운 섬나라 입니다.마라도소재지 : 서귀포시 대정읍 마라리 운진항 ( 기점11km, 가파도 기점 5.5km)크기 : 동서길이 500m, 남북길이 1.3km, 섬둘레 4.2km보호구역 : 전체면적의 85%, 천연기념물 제423호 (지정일 2000년 7월)볼거리 : 대한민국 최남단비, 가파른 절벽과 해식동굴, 제주도 남서쪽 해안 전망으로 산방산, 형제섬, 한라산 등이 어우러진 광경, 장군바위, 대문바위, 애기업개당 등 자연화산 생서울기타시설물 : 태양광발전소, 등대, 기원정자, 마라분교 등가파도소재지 : 서귀포시 대정읍 하모리 (운진항에서 남쪽으로 2.2km 해상)크기 : 해안선길이 4.2km, 최고점 20.5m볼거리 : 소망전망대, 할망당(상동,하동), 돈물깍, 보름바위(큰왕돌), 까마귀돌(동산), 고냉이돌, 고인돌군락지 등먹거리 : 해산물정식, 보말칼국수, 짬뽕, 성게국, 가파도핫도그 등자전거체험 : 대여가능


no 105 :  제주목관아
제주 제주시
제주특별자치도 제주시 관덕로 25
조선시대 제주지방 통치의 중심지였던 제주목 관아(濟州牧 官衙)는 지금의 관덕정을 포함하는 주변일대에 분포해 있었으며, 이미 탐라국시대부터 성주청 등 주요 관아시설이 있었던 곳으로 추정되고 있다. 관아시설은 1434년 관부의 화재로 건물이 모두 불 타 없어진 뒤 바로 역사를 시작하여 그 다음해인 1435년에 골격이 이루어졌으며, 조선시대 내내 중.개축이 이루어졌다. 그러나 제주목 관아는 일제강점기 때 집중적으로 훼철되어 관덕정을 빼고는 그 흔적을 볼 수가 없었다. 제주시에서는 탐라국 이래 조선시대에  이르기까지 제주의 정치.행정.문화의 중심지 역할을 해왔던 제주목 관아를 원래의 양식으로 복구하고자,  1991년부터 1998년까지 4차례 발굴조사를 마친 결과, 탐라시대부터 조선시대에 이르기까지의  여러 문화층과 함께 문헌상에 나타

no 119 :  선임교
제주 서귀포시
제주특별자치도 서귀포시 중문로105번길 37
천제연 폭포 위쪽에 있는 아치형의 선임교는 칠선녀다리라고도하며, 국내최초로 고유의 오작교 형태로 건설되었고 양쪽면에 각각 다른 악기를 든 선녀들이 구름을 타고 하늘로 올라가는 아름답고 웅장하게 조각된 일곱선녀상이 새겨져 있다. 구름다리, 선녀다리, 칠선녀다리, 칠선녀교, 선임교 등으로 불리워지는 것 만큼이나 유명하다. 칠선녀교라고도 하는 선임교(仙臨僑)는 천제연의 2단과 3단폭포 중간쯤에 위치해 폭포와 중문관광단지를 이어주는 아치형 철제다리이다.국내 최초로 우리 고유의 오작교 형태로 꾸며놓은 선임교 양쪽 옆면에는 칠선녀의 전설을 살려 각각 다른 악기를 든 아름답고 웅장하게 조각된 일곱 선녀상이 있으며, 한 개의 길이가 20미터나 되는 14개의 선녀상은 방문객들의 눈길을 끈다. 다리 길이는 128m이고 폭4m로 230톤 무게를 견딜 수 있다. 또한, 야간 관광에 대비하여 1백개 난간사이에 34개의 석등이 설치되어 있다.


no 120 :  산천단곰솔
제주 제주시
제주특별자치도 제주시 516로 3041-24
1964년 1월 31일 천연기념물 제160호로 지정되었다. 대흥사에서 소유하고 있다. 곰솔은 높게 자라는 소나무과의 상록침엽교목으로 흔히 해송, 검솔 또는 나무껍질이 검은빛을 띤다 하여 흑송이라고도 부른다. 한국에서는 소나무와 함께 쌍유관속아속에 포함되는데 서쪽으로 경기도, 동쪽으로 강원도 명주군까지 바닷가를 따라 분포하고 있으며, 주 분포지역은 남부지방과 제주도의 해발 700m 이하이다.내한성이 약하기 때문에 중부 내륙지방이나 깊은 산속에서는 자라지 못한다. 수피는 흑갈색이고 가지 끝에 형성되는 겨울눈의 색깔이 희어서 붉게 보이는 소나무와 쉽게 구별이 된다. 잎은 짙은 녹색을 띠는데 약간 비틀리고 끝이 뾰족하며 길이 9∼14㎝이고 2개씩 잎집에 모여 난다.제주시의 곰솔은 제주시에서 서귀포시로 향하는 제1횡단도로를 따라 8㎞ 정도 가면 나타나는 산천단이라는 곳에 군락을 이룬다

no 132 :  수목원테마파크(아이스뮤지엄)
제주 제주시
제주특별자치도 제주시 은수길 69
수목원테마파크(아이스뮤지엄)에는 다양한 얼음 조각전시와 관람객들이 직접 체험할 수 있는 이글루, 대형 아이스 미끄럼틀로 더욱 재미있는 공간을 연출하였다.아이들이 더욱 좋아하는 5D영상관과 그림이 살아 움직이는 증강현실 착시아트가 있어 더욱 실감나는 AR증강현실 체험을 할 수 있다. 그 외 플레이박스(가상현실체험)와 수목원길(LED공원) 다양한 식당가들이 있다.


no 133 :  성읍민속마을
제주 서귀포시
제주특별자치도 서귀포시 표선면 성읍정의현로 19
※ 19_20 한국관광 100선 ※표선리에서 북쪽으로 8km쯤 가면 한라산 기슭 아래 성읍마을이 있다. 성읍리는 원래 제주도가 방위상 3현으로 나뉘어 통치되었을 때(1410-1914) 정의현의 도읍지였던 마을로서 제주도 옛 민가의 특징을 잘 간직하고 있는 곳이다. 유형, 무형의 많은 문화유산이 집단적으로 분포되어 있고, 옛마을 형태의 민속경관이 잘 유지되어 그 옛 모습을 계속 유지하고자 민속마을로 지정.보호하고 있다. 이곳에서는 옛민가, 향교, 옛관공서, 돌하루방, 연자방아, 성터, 비석 등의 유형 문화유산과  중산간 지대 특유의 민요, 민속놀이, 향토음식, 민간공예, 제주방언 등의 무형 문화유산이 아직까지 전수되고 있다. 천연기념물 제 161호인 느티나무, 팽나무 등은 오백년 도읍지로서의 긍지를 엿볼 수 있다. 제주 초가를 가장 잘 보존하고 있는 성읍민속마을은 실제로 제주 초가에 살고 있는 성읍리 주민들과 제주의 전통 화장실인 ''통시''를 만나볼 수도 있다. 특히 마을내 고목과 돌담 그리고 옛 성벽 등이 어우러진 고풍스런 모습은 오백년 도읍지의 역사를 그대로 말해준다.


no 134 :  더마파크
제주 제주시
제주특별자치도 제주시 한림읍 월림7길 155
더馬파크는 세계 최초의 말 전문 테마공원이다.승마와 기마공연 등 말의 고장 제주도에서 말에 관한 모든 것을 즐길 수 있게 한 명품 테마공원이다.2008년 첫 공연

no 147 :  주상절리대(대포동지삿개)
제주 서귀포시
제주특별자치도 서귀포시 이어도로 36-30
막혔던 가슴이 탁트이도록 시원스레 부서지는 파도, 신이 다듬은 듯 정교하게 겹겹이 쌓은 검붉은 육모꼴의 돌기둥이 병풍처럼 둘러쳐져 있는 중문관광단지 동부지역 해안가의 주상절리대는 자연의 위대함과 절묘함을 동시에 느낄 수 있는 천혜의 관광자원으로, 제주도 지정문화재 기념물 제50호다. 아득한 옛날 지각변동으로 인해 이루어진 주상절리대를 보고 있으면 새하얗게 부서지는 포말 속에 석수장이의 애달픈 사연이라도 금새 실려오는 듯하다. 파도가 심하게 일때는 높이 20m 이상 용솟음 치는 장관을 연출한다. 천혜의 절경으로 인해 테마여행지로 각광받고 있는 명소다.* 주상절리(柱狀節理, columnar jointing)란 주로 현무암질 용암류에 나타나는 기둥모양의 수직절리로서 다각형(보통은 4∼6각형)이며, 두꺼운 용암(약 섭씨 1100도)이 화구로부터 흘러나와 급격히 식으면서 발생하는 수축작용의 결과로서 형성된다고 하는데 이곳의 주상절리는 높이가 30~40m, 폭이 약 1km 정도로 우리나라 최대 규모이다. * 지질학적으로는 주상절리지만 행정구역으로는 서귀포시 중문동이며 이곳의 옛이름인 "지삿개"를 살려 "지삿개바위"로 부른다.


no 148 :  새별오름
제주 제주시
제주특별자치도 제주시 애월읍 봉성리 산
남봉을 정점으로 남서, 북서, 북동방향으로 등성이가 있으며, 등성이마다 봉우리가 있다. 서쪽은 삼태기모양으로 넓게 열려있고, 북쪽은 우묵하게 패여 있다. 마치 별표처럼 둥그런 표창같은 5개의 봉우리가 존재한다. 전체적으로 풀밭을 이루나 북쪽사면에 일부 잡목이 형성되어 있고, 서북쪽 사면에 공동묘지가 조성되어 있다. 저녁하늘에 샛별과 같이 외롭게 서 있다하여 붙여진 이름이다. * 표고 519.3m /  비고 119m / 둘레  2,713m /  면적  522,216㎡  / 저경  654m


no 149 :  수월봉
제주 제주시
제주특별자치도 제주시 한경면 고락로
제주의 

In [3]:
#xPath로 게시글 수 가져온 것,, 먹힐까?

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt
from selenium.common.exceptions import NoSuchElementException
import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '관광지 서울부산인천'
# 파일 저장
f_dir = 'c:\\캡디_공항인근\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EA%B4%80%EA%B4%91%EC%A7%80&choiceTagId=3f36ca4b-6f45-45cb-9042-265c96a4868c')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
view2 = []      # 게시글 내용 컬럼


no = 1

for j in range(0,4):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()

    time.sleep(3)

    r_cnt= driver.find_element_by_xpath('''//*[@id="totalCnt"]''').text
    r_cnt2 = r_cnt.replace(",","")
    search_cnt = int(r_cnt2)
    
    cnttt = 150
    page_cnt = math.ceil(cnttt/10)

    if cnttt > search_cnt :
        cnttt = search_cnt

    print("전체 검색 결과 건수 :",search_cnt,"건")
    print("실제 최종 출력 건수",cnttt)
    
    for i in range(1,page_cnt+1):
        
        for k in range(1,11):
            f = open(ff_name, 'a', encoding='UTF-8')
#             if no > cnttt :
#                 break
                
            #각 게시글의 제목 누르기    
            title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
            driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
            time.sleep(random.randrange(5,7))

            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')        
            content_list = soup.find('div','wrap_contView')    
            con_1 = content_list.find('p').get_text()
            try:
                area_CD_tmp = soup.find('div', class_='area_address')
                area_CD = area_CD_tmp.find('span').get_text()
            except AttributeError:
                area_CD = '0'
            except:
                area_CD_tmp = soup.find('div', {"id" : "topAddr"})
                area_CD = area_CD_tmp.find('span').get_text()
            
            try:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            except IndexError:
                try:
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                except IndexError:
                    addr = '0'
                except NoSuchElementException:
                    addr ='0'
            except NoSuchElementException:
                try:
                    addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                    addr = addr.split('주소 ')
                    addr = addr[1]
                except NoSuchElementException:
                    addr = '0'
            except:
                addr = '0'
            
            
            print("no", no, ": ", title)
            print(area_CD)
            print(addr)
            print(con_1)
            f.write(str(no) + ';'+ str(title) + ';'+str(area_CD) +';'+ str(addr) +';'+ str(con_1) + "\n")
            f.close( )

            print("\n")
            no += 1
            no2.append(no)
            contents.append(title)
            area_CD2.append(area_CD)
            addr2.append(addr)
            view2.append(con_1)
            
            driver.back( )  # 뒤로 돌아가기 기능
            time.sleep(random.randrange(5,7))
            
            if no > cnttt:
                break
        if i > page_cnt+1:
            break
        i += 1
        
        if no > cnttt:
            break
        
        if (i % 5 == 0):
            driver.find_element_by_link_text('''다음''').click()
        else :
            driver.find_element_by_link_text("""%s""" %i).click() # 다음 페이지번호 클릭    
        time.sleep(2)
    i=0
    k=0
    no=1
        
a = pd.DataFrame()
a['no']=no2
a['title']=contents
a['area_cd']=area_CD2
a['addr']=addr2
a['content']=view2

# csv 형태로 저장하기
a.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
a.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

전체 검색 결과 건수 : 562 건
실제 최종 출력 건수 150
no 1 :  남산골한옥마을
서울 중구
서울특별시 중구 퇴계로34길 28
1998년 조성된 남산골한옥마을은 남산 북측 옛 수도방위사령부 부지 총 79,934m²(24,180평)에 한옥 5동, 전통공예관, 천우각, 전통정원, 서울남산국악당, 새천년타임캡슐 광장으로 구성되어 시민과 관광객을 맞이하고 있다. 번화한 도심 한 가운데에서 한국 전통문화를 소재로 한 다양한 체험거리를 만나볼 수 있으며, 남산 자락을 따라 전통조경 양식으로 조성된 계곡과 정자, 각종 화초가 만들어내는 아름다운 풍경을 만날 수 있다. 전통정원 가장 높은 곳에는 서울정도 600년을 기념하고 400년 후 서울정도 천년을 기대하며 1994년 당대의 기억을 매설한 서울천년타입캡슐 광장이 조성되어 있다. 옛 가옥을 복원해 놓은 남산골한옥마을 남산골한옥마을 안으로 들어서면 연못 청학지와 그 곁으로 천우각이 있다. 천우각 광장을 지나 돌계단을 오르면 서울 곳곳에 있던 한옥이 이전·복원되어 옛 사람들의 생활방식을 볼 수 있게 한 예스러운 공간이 나타난다.모두 다섯 채의 한옥으로서 삼각동 도편수 이승업 가옥, 삼청동 오위장 김춘영 가옥, 관훈동 민씨 가옥, 제기동 해풍부원군 윤택영 재실, 옥인동 윤씨 가옥이 있다. 이 중 건물 이전이 불가능할 정도로 낡은 옥인동 윤씨 가옥만 새 자재를 사용해 복원했고 나머지 건물은 종래의 집 부재를 그대로 이전했다. 남산골한옥마을에서 즐기는 다양한 체험거리 남산골한옥마을에는 다양한 체험거리가 마련되어 있다. 한옥 다섯 채를 활용하여 한옥실내 공간을 체험하면서 동시에 한국 전통문화프로그램을 접할 수 있다. 한복 입기, 한지 접기, 한글 쓰기, 전통차 마시기를 체험할 수 있고, 전통예절학교와 한방체험도 운영하고 있다. 토요일과 일요일 양일에는 전통혼례가 실제로 치러지고 전통혼례체험도 운영하고 있다. 그 밖에도 천우각 맞은편 원두막에서는 짚공예 시연이 이루어지고 있으며, 천우각 무대에서는 태권도 시범공연과 체험 행사

no 11 :  덕수궁
서울 중구
서울특별시 중구 세종대로 99
※ 코로나바이러스감염증-19 공지사항※ 내용 : 석조전, 중명전 임시휴관(2020.08.19 ~ 상황 종료시)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기※ 19_20 한국관광 100선 ※사적 제124호의 덕수궁은 본디 왕궁이 아니었다. 원래 성종의 형인 월산대군(1454∼1488)의 집이었으나, 1592년 임진왜란 때 왕궁이 모두 불타서 1593년 행궁으로 사용하기 시작하였다. 선조임금은 의주로 피난갔다가 한양으로 환도한 뒤 덕수궁에서 머무른다. 선조의 뒤를 이은 광해군은 1608년 이곳 행궁에서 즉위한 후 1611년 행궁을 경운궁이라 고쳐부르고 7년동안 왕궁으로 사용하다가 1615년에 창덕궁으로 옮기면서 이곳에는 선왕인 선조의 계비인 인목 대비만을 유폐시켰다. 1618년에는 인목대비의 존호를 폐지하고, 경운궁을 서궁이라 낮추어 부르기도 하였다. 1623년 인조반정으로 광해군이 폐위되고 인조가 이곳 즉조당에서 즉의하고 창덕궁으로 옮긴 이후 270년 동안은 별궁으로 사용되었다. 1897년 고종황제가 러시아 공관에 있다가 환궁하면서 이곳을 다시  왕궁으로 사용하게 되었는데, 그때부터 다시 경운궁이라 부르게 되었고 그 규모도 다시 넓혔다. 그리고 고종 황제는 1907년 순종에게 양위한 후, 왕궁을 창덕궁으로 옮긴 후에도 이곳에 거처하였는데, 이때부터 고종 황제의 장수를 비는 뜻에서 덕수궁이라 부르게 되었다. 이런 까닭에 요즘와서는 경운궁이라는 본래의 명칭을 사용해야 한다는 주장도 제기되고 있다.덕수궁에서 지난 날 많은 건물이 있었으나 현재  18,635평의 경내에 남아 있는 것은 대한문,중화전, 광명문, 석어당, 준명당, 즉조당, 함녕전, 덕홍전 및 석조전 등에 불과하다. 덕수궁은 특히 구한말 비운의 황제 고종이 일제의 압박으로 양위를 강요당하고, 한 많은 여생을 보내다가 1919년 1월 22일 덕수궁 함녕전에서 일제에 의해 독살당하여 돌아가신 것으로 알려져 3·1 독립운동의 직접적인 계기가

no 24 :  서울 암사동 유적
서울 강동구
서울특별시 강동구 올림픽로 875
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관(2020.08.19.~)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기지난 1925년 한강대홍수로 처음 유적지가 발견된 이래, 여러 차례의 발굴 과정을 거쳐 1979년 7월 26일, 국가지정문화재 사적 제 267호로 지정된 암사동 선사주거지는 우리나라에서 발견된 신석기시대 사람들의 최대 집단취락지이다. 전체 넓이 23,600여평의 대지 위에 잔디 및 나무가 우거져 있으며 556평 규모의 2개소의 원시생활전시관, 그리고 야외에 9개의 움집 및 관람객이 직접 들어가서 신석기시대 사람들의 생활상을 직접 볼 수 있는 1개소의 개방형 "체험움집"이 설치되어 있다. 원시생활 전시관의 주요 전시 내용은 6000년 전의 집터 8개, 빗살무늬토기, 석기, 돌도끼 등 신석기시대의 유물 및 선사시대 영상 방영을 위한 50석 규모의 영상실, 신석기시대 사람들의 생활상에 대한 각종 입체모형, 발굴 당시의 현장 축소 모형, 자동 작동 62인치 3D영상 프로젝션TV, 자료검색용 터치스크린 컴퓨터, 직접 실습할 수 있는 불피우기도구, 기념사진촬영을 위한 원시인 모형 등이다. 개방형 "체험 움집"에는 29평 규모에 창을 손질하고 있는 아버지의 모습, 돌칼로 고기를 써는 어머니의 모습, 물고기를 굽는 아들의 모습, 음식을 먹고 있는 딸의 모습 및 당시 음식물을 보관해 두었던 저장고 시설, 자동감지설명 나레이션이 설치되어 있어 관람객들의 신석기시대 사람들의 생활상에 대한 이해를 돕고 있다. 이 암사동 유적지는 우리 나라에서 가장 잘 정비된 신석기시대의 유적지이며, 특히 출토 유물 중 신석기시대의 가장 대표적인 토기인 빗살무늬토기가 유명하다.* 시대 - 석기시대


no 25 :  아리수 정수센터 (암사,뚝섬,영등포)
서울 강동구
서울특별시 강동구 아리수로 131
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관(2020.08.19.~)→ 코로나

no 34 :  서울 경교장
서울 종로구
서울특별시 종로구 새문안로 29
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.19 ~ 상황 종료시)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기사적 제465호 경교장은 대한민국 임시정부의 활동공간이자, 백범 김구 선생이 서거한 역사의 현장이다. 서울시는 대한민국임시정부의 마지막 청사이자 백범 김구가 서거한 역사적 현장인 경교장을 원형 복원하고, 우리 근현대사의 역사적 현장을 되살려 시민들의 교육공간으로 활용하고자 경교장을 복원하였다. 뿐만 아니라 내부를 임시정부의 지나온 역사를 한눈에 조망할 수 있도록 전시공간으로 꾸며 ''13. 3. 2(토) 부터 개방하고 있다.경교장 복원공사는1949년 백범 김구 서거 이후 오랜 기간 대사관 및 병원시설로 사용되면서 변형된 내부 평면을 철거하고, 이 과정에서 옛 모습이 잘 남아있는 부분은 최대한 원형을 살려 복원하였다. 그렇지 않은 부분은 『조선과 건축(朝鮮と建築)』(1938년 8월호)에 수록되어 있는 경교장 도면을 근거로 하여 복원하였다. 복원된 경교장에서는 대한민국 임시정부가 걸어온 길을 유물과 영상, 정보검색코너 등 다양한 콘텐츠를 통해 살펴볼 수 있다.


no 35 :  어린이 감각놀이터 디키디키
서울 중구
서울특별시 중구 을지로 281
※ 코로나바이러스감염증-19 공지사항※ 내용 : 임시휴관 (2020.08.25 ~ 상황 종료시)→ 코로나바이러스감염증-19 여행정보 변동사항 확인하기''Design Kids Digital Kids''를 뜻하는 디키디키는 디자인을 통한 어린이의 감성과 창의력을 키우는 공간으로, 유·아동기부터 문화, 예술, 산업 전반의 디자인 마인드를 자연스럽게 체득 할 수 있는 디자인 활동이 가능한 대규모 놀이터이다.이 곳에서 아날로그 놀이와 디지털 매체를 융합환 디지로그 컨셉의 새롭고 건강한 놀이문화를 마음껏 즐길 수 있다.


no 36 :  서울 애니메이션센터
서울 중구
서울특별시 중구 소공로 48
※ 코로나바이러스

no 54 :  양재 시민의숲
서울 서초구
서울특별시 서초구 매헌로 99
1986년의 서울아시아경기대회와 1988년의 제24회 올림픽경기대회를 위해 서울특별시의 관문인 양재 톨게이트 주변에 조성한 공원으로, 다양한 수종의 숲이 볼만하며(소나무, 느티나무, 당단풍, 칠엽수, 잣나무 등 43종 94,800주), 숲이 좋아 연인들의 만남의 장소로 이용되고 있다. 현재 도심에서는 매우 보기 드문 울창한 수림대를 형성하고 있고, 특히 가을에는 감, 모과 등 과일이 열려 풍성한 자연을 만끽할 수 있다. 야외예식장과, 농구장, 배구장(족구장 겸용), 테니스장, 맨발공원 등의 운동시설이 있으며, 윤봉길의사 상, 윤봉길의사 숭모비, 대한항공 위령탑, 삼풍사고위령탑, 유격백마부대충혼탑이 있다.


no 55 :  길동생태공원
서울 강동구
서울특별시 강동구 천호대로 1291
서울시에서 조성한 유일한 자연생태공원으로서 강동구 천호대로변 습지에 생물 서식공간을 복원하여 시민과 학생들에게 자연생태의 학습과 자연체험공간을 제공하며 생물의 다양성을 증진시켜 도시민들에게 건강한 녹지 공간 제공으로 도시환경의 질을 높이기 위하여 조성된 공원이다.               전체 80,683㎡(24,407평)에 습지지구, 산림지구, 초지지구광장지구를 조성하였으며 야외 강의장, 야외 전시장, 야외 관찰대, 파고라, 조류 관찰대, 수중섬, 자연초지와 초가집, 움집, 인공벌통, 돌담, 버섯재배대, 새집, 조류먹이공급대 등이 설치 조성되어 있다. 특히, 조류 관찰대는 나무로 높이 칸막이를 만들고 사방 30cm 정도의 관찰창을 뚫어 새들이 놀라지 않게 세심하게 배려하였다. 계절에 따라 물총새, 왜가리, 원앙, 흰뺨검둥오리, 꼬마물떼새등을 관찰할 수 있다. 습지지구에서는 수생식물과 수서곤충 개구리 등을 볼 수 있고, 수련 노랑어리 연꽃 갈대창포와 하늘의 사냥꾼 잠자리, 물방개 물자라 등을 볼 수 있다.


no 56 :  여의도샛강생태공원
서울 영등포구
서울특별시 영등포구 여의동로 48
여의도샛강 생태공원

...

no 77 :  팔미도 유람선
인천 중구
인천광역시 중구 연안부두로 36
우리나라 최초의 등대가 있는 팔미도, 1903년 6월 1일 우리나라에서 최초로 등대 불빛을 밝힌 팔미도 등대는 지난 100년간의 임무를 수행하고 인천시 지방문화재(제 40호)로 지정되어 그 자리를 지키며 보존되고 있고 현재는 새로 지은 등대가 불빛을 비추고 있다. 이 곳 팔미도는 원래 사람들의 발길이 통제되어 있었지만 2009년 1월 1일부터 입도가 허락되었다.


no 78 :  영흥도
인천 옹진군
인천광역시 옹진군 영흥면
영흥도는 영흥대교와 선재대교를 통해 육지와 연결되어, 관광객이 줄을 잇고 있으며 무진장한 수산어업 전진기지로서 눈부시게 발전하고 있다. 인천에서 남서쪽으로 32km 거리에 있는 섬으로 옹진군에서 백령도 다음으로 큰 이 섬은 영흥도와 선재도, 측도 등 유인도서 4개와 외항도, 중도, 자암도 등 무인도서 18개를 포함하여 연근해에 광활하게 펼쳐져 있다. 해안경관이 수려하고 노송지대가 자리하고 있어 시원한 그늘공간을 제공해 주고 있으며 백사장이 널리 펼쳐져 있어 일출 일몰이 경치와 어우러져 아름다운 경관을 이룬다.


no 79 :  용유도
인천 중구
인천광역시 중구 마시란로 308-13
면적 13.603㎢, 해안선 길이 48.2km의 섬으로, 영종도와 연륙교로 연결되어 있었으나 인천국제공항 건설을 위하여 부근의 신불도(新佛島), 삼목도(三木島), 영종도(永宗島)와 함께 간척사업으로 하나가 되었다. 주변 섬으로는 대무의도(大舞衣島), 장봉도(長峰島), 모도(茅島), 시도(矢島), 신도(信島) 등이 있다. 용유도에는 1986년 국민관광지로 지정된 을왕리해수욕장을 비롯하여 왕산 해수욕장, 선녀바위 해수욕장, 마시안 해변 등이 서남쪽 해안을 따라 조성되어 있으며, 매년 8월초 해양축제와 물고기잡기 대회, 어패류 채취 등 온 가족의 자연학습장으로 좋다.또한 2001년 인천국제공항개항과 더불어 국내 최초로 해상관광호텔이 들어서고 카지노, 호텔, 위락단지, 골프장 등 천혜의 자연 경

no 94 :  동인천 삼치거리
인천 중구
인천광역시 중구 우현로67번길 57
동인천역 앞 인천학생문화회관 옆 골목이 삼치골목이다. 이골목은 지금부터 40여 년 전 ''인하의 집'' 이라는 식당이 생기면서 시작됐다. 당시 인하의 집은 가정집에서 손님을 받았다. 또한 지금 삼치골목이 아니라 그 뒷골목에 인하의 집이 있었다. 지금처럼 식당을 차린 것은 30년이 약간 못 된다. 그때부터 지금의 삼치골목 거리에 삼치집이 하나 둘씩 생기기 시작했다. 삼치골목도 처음부터 삼치를 구워냈던 것은 아니다. 다른 여러 안주 가운데 삼치가 인기를 끌어서 삼치가 대표메뉴가 됐던 것이다. 이렇게 해서 삼치골목으로 자리잡게 됐다. 지금 삼치골목 인하의집 근처 벽면에 보면 처음 이 거리에서 삼치를 구웠던 인하의집 할머니와 아주머니가 벽화에 등장한다. 그 두 분의 힘으로 삼치골목이 지금처럼 꾸며지게 된 것이다. 십여 집이 이 골목에서 성업중이다. 2002년에는 구에서 삼치구이거리로 지정하여 골목 입구에 입간판을 세우기도 했다. 지금은 예전의 대문짝만한 간판을 다 없애고 각 식당마다 특색 있고 예쁜 간판으로 다 바꿨다. 간판이 바뀌니 골목 분위기도 그럴듯 하게 바뀌었다. 이 골목 삼치는 뉴질랜드 산이다. 삼치를 구워내는 방식과 찍어 먹는 소스에 따라 약간의 맛 차이가 난다. 또한 삼치 이외의 안주도 집 마다 많이 내놓고 있다.


no 95 :  용궁사(인천)
인천 중구
인천광역시 중구 운남로 199-1
지금부터 1,300여 년 전인 신라 문무왕 10년(670년)에 원효대사가 창건하여 산 이름을 백운산, 절 이름을 백운사라 하였다. 흥선대원군이 이 절에 머물면서 10년 동안 기도를 했다. 1864년에 아들이 왕위에 오르자, 흥선대원군은 절을 옛터에 옮겨 짓고, 구담사에서 용궁사로 이름을 바꾸었다. 일설에는 영종도 중산 월촌에 사는 윤공이란 어부가 꿈을 꾼 뒤 바다에서 작은 옥불을 어망으로 끌어올려  이 절에 봉안했다는 전설에 따라 붙여진 이름이라고도 한다.  현재 대원군이 중건한 관음전과 

no 111 :  강화 덕진진
인천 강화군
인천광역시 강화군 불은면 덕진로 34
고려시대 강화해협을 지키는 외성의 요충으로서 원래 수영에 예속된 첨사의 진(鎭) 이었으며, 1677년 만호를 두고 군관 26명, 병 100명, 돈군 12명, 배 2척이 배치되었다. 1679년 축조된 용두, 덕진돈대 포좌 15문이 설치된 강화 제 1의  포대인 남장포대와 대포 10문이 설치된 덕진포대가 이 진에 소속되었다.1866년 병인양요 때는 양헌수 장군의 부대가 밤에 이 진을 통하여 정족산성에 들어가 승리를 거두었고, 1871년 신미양요 때 미국의 아시아 함대와 가장  치열한 포격전을 전개하여 미국 함대에 피해를 입히기도 했다. 당시 미군의 전사 속에 남북전쟁 때에도 이와 같이 짧은 시간에 맹렬한 포격전을 경험하지 못했다고 기록하고 있다. 신미양요 때 성첩과 문루가 모두  파괴되었던 것을 1977년에 복원하면서 남장포대에는 조선군이 사용하던 대포를 제작하여  설치하였다.* 소유자 - 국유* 규모 - 1기


no 112 :  강화산성
인천 강화군
인천 강화군 강화읍 국화리 산3번지
강화읍은 고려시대 몽골이 침입했을 때 잠시 도읍을 옮겼던 곳으로 이와 관련한 유적들이 많다. 강화산성 역시 그같은 문화유산으로 고려시대에 쌓은 산성이다. 강화산성은 고려 고종 19년(1232)에 몽골의 제2차 침입에 대항하고자  착공되었다. 공사가 완공되기도 전에 강화도로 천도하는 사태가 발생했고 1234년부터 본격적으로 축성되었다.  고려의 성은 내, 외, 중성이 있었는데 내성은 현재의 강화성으로 둘레가 7,122m이고 중성은 둘레가 5,381m로 1250년에 축성하였다.  외성은 1233년에 축조되기 시작했다. 이 성들은 모두 토성이었으나 1677년에 강화 유수 허질이 대대적인 개축을 하면서 석성으로 쌓았다. 1709년에 강화유수 박권이 다시 개축했고 조선 후기에 보수가 이어졌다. 강화산성에는 안파루(남), 첨화루(서), 망한루(동), 진송루(북)의 사대문과 암문, 수문, 장대 등의 방어시설이 있

no 128 :  연무당 옛터
인천 강화군
인천광역시 강화군 강화읍 국화리
강화산성 서문 건너편에 있는 연무당은 1876년 강화도조약이 최종 조인된 장소로 이 곳에서 체결된 강화도조약에 의해 우리나라는 부산, 인천, 원산을 일본에게 개항하였다.고종 7년에 창건된 연무당은 본시 지금의 상 시장 군 농협 자리 서쪽에 있었는데, 비좁아 동소문 밖으로 옮겼다가 내성서문 남쪽에 당을 세우고 그 전면에 조련장이 마련되어 있었다. 이 연무당은 병자수호조약을 체결한 곳으로서 우리 민족의 암울했던 과거사를 잊지않기 위해 이 곳에 기적비가 세워져, 민족 자주의식을 드높이 간직해야 한다는 경고를 하고 있다.


no 129 :  잠진도
인천 중구
인천광역시 중구 잠진도길 144
인천 중구 위치한 잠진도는 영종도에서 연륙도로로 이어져 있는 작은 섬이다. 밀물 때 물이 차오르면 섬이 잠길 듯 말듯 한다 하여 잠진도라고 이름이 붙여졌다. 갯벌을 붉게 물들이는 낙조가 아름답기로 유명하여 사진촬영을 하려고 찾아오는 관광객이 많으며 물이 빠진 뒤에 갯벌에서 조개를 캐는 사람들도 많다. 또 무의대교 개통으로 차량으로 이동할 수 있다. 주변 가까운 곳에는 을왕리해수욕장, 덕교해수욕장, 하나개해수욕장 등이 있다.


no 130 :  강화 참성단
인천 강화군
인천광역시 강화군 화도면 흥왕리
참성단은 단군이 하늘에 제사를 올리던 제단이라고 전한다. 제단은 돌로 하부는 둥글게, 그 위에 네모난 단을 쌓아 올렸다.  타원형의 하부는 13m*17.5m이며 지름은 8.7m이며, 네모난 단은 6.6m의 정방형이다. 고려 원종 11년(1270)에 보수하였으며, 조선 인조 17년(1639)에 수축하였고, 숙종26년(1700)에도 보수하였다. 지금은 해마다 개천절에 이곳에서 단군의 제사를 지내며,  한편 전국체육대회 때마다 대회장에 타오르는 성화가 이 참성단에서 7선녀에 의해 채화되어 대회장으로 봉송, 점화되고 있다. 특히 1988 세계장애자 올림픽대회의 성화도 이곳에서 채화되어 대회 기간을 밝힌 바 있다.* 

no 146 :  인천항크루즈터미널
인천 연수구
인천광역시 연수구 국제항만대로 438
2019년 개장한 인천항 크루즈터미널은 국내최대규모로 22만 5천 톤급 초대형 크루즈선이 접안할 수 있는 부두와 연면적 약 6,566㎡ 규모의 2층 터미널을 갖추고 있다. 또한 대형버스 156대를 포함하여 200여 대가 동시에 주차할 수 있는 넓은 주차장도 마련되어 있어 크루즈 여행객들이 편리하게 이용할 수 있다.


no 147 :  인천대공원 반려동물 놀이터
인천 남동구
인천광역시 남동구 무네미로 236
반려견과 반려인이 마음껏 뛰어놀 수 있는 반려동물 놀이터가 2018년 7월 2일(월) 인천대공원에 문을 열었다. 이는 인천에 처음으로 조성된 반려동물 놀이터로 대형견과 중·소형견 놀이터가 분리되어 있으며, 흥미로운 공간으로 꾸며졌다. 또한 대형파라솔 및 야외탁자 등을 설치해, 반려동물을 동반한 이용자들도 휴식과 놀이를 함께 즐길 수 있도록 했다.


no 148 :  강화해수랜드
인천 강화군
인천광역시 강화군 길상면 해안남로 13-12
강화해수랜드는 강화초지대교 옆에 위치해 있으며, 1천3백평으로 국내 최대의 규모를 자랑한다. 해수탕의 특징은 해양 암반심층수로 지하 560M에서 용출되고 암반층에 걸러진 물이라 깨끗한 물이다.강화해수랜드의 해수는 인체가 생명을 유지하는데 없어서는 안되는 미네랄이 골고루 함유되어 있으며 몸과 해수의 염도 차이를 이용한 삼투압 작용으로 인하여 몸속의 노폐물은 쉽게 배출되고 필요한 광물질은 효과적으로 흡수되어 신진대사, 혈액순환을 도우며  살균, 해독작용으로 인하여 관절염 신경통,아토피성 피부질환 등 피부미용에 탁월한 효과가 있다. 또한 해수로 입안을 헹구면 잇몸질환, 충치 등 각종 구강질병까지 예방할 수 있다.


no 149 :  대한민국 수준원점
인천 미추홀구
인천광역시 미추홀구 인하로 100
수준원점이란 우리나라의 국토 높이를 측정하는 기준점으로, 평균 해수면과의 차이를 측량하여 1963년 육지에 설치해 놓은 시설물이다. 전국의 수준점 및